In [ ]:
import os

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

import networkx as nx

import statsmodels.api as sm
from statsmodels.tsa.api import VAR
from statsmodels.tsa.vector_ar.vecm import coint_johansen
from statsmodels.tsa.stattools import coint

from sklearn.preprocessing import MinMaxScaler

from prophet import Prophet

from scipy.stats import pearsonr

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller, grangercausalitytests, kpss

import pymannkendall as mk

In [ ]:
data_folder = os.path.join("..", "..", "data", "berlin")
plot_folder = os.path.join(data_folder, "plots")
surface_plot_folder = os.path.join(plot_folder, "surface")
surface_cross_corr_folder = os.path.join(surface_plot_folder, "cross_corr")
surface_corr_folder = os.path.join(surface_plot_folder, "corr")

ground_plot_folder = os.path.join(plot_folder, "ground")
ground_cross_corr_folder = os.path.join(ground_plot_folder, "cross_corr")
ground_corr_folder = os.path.join(ground_plot_folder, "corr")

clean_data_folder = os.path.join(data_folder, "clean_data")
projections_folder = os.path.join(data_folder, "projections")

my_projections_folder = os.path.join(projections_folder, "found_by_me")
cat_projections_folder = os.path.join(projections_folder, "cat")

paper_plots_folder = os.path.join("..", "..", "paper plots")

# Load Clean Data

In [ ]:
surface_df = pd.read_excel(
    os.path.join(clean_data_folder, "surface.xlsx")
)
ground_df = pd.read_excel(
    os.path.join(clean_data_folder, "ground.xlsx")
)

flow_river_projections = pd.read_excel(
    os.path.join(my_projections_folder, "flow_river.xlsx")
)

hist_flow_river_projections = pd.read_excel(
    os.path.join(my_projections_folder, "flow_river_hist.xlsx")
)

air_temp_projections = pd.read_excel(
    os.path.join(my_projections_folder, "air_temp.xlsx")
)

hist_air_temp_projections = pd.read_excel(
    os.path.join(my_projections_folder, "air_temp_hist.xlsx")
)

precip_projections = pd.read_excel(
    os.path.join(my_projections_folder, "precip.xlsx")
)

hist_precip_projections = pd.read_excel(
    os.path.join(my_projections_folder, "precip_hist.xlsx")
)

cat_flow_river_projections = pd.read_excel(
    os.path.join(cat_projections_folder, "flow_river.xlsx")
)

cat_air_temp_projections = pd.read_excel(
    os.path.join(cat_projections_folder, "air_temp.xlsx")
)

cat_precip_projections = pd.read_excel(
    os.path.join(cat_projections_folder, "precip.xlsx")
)

In [ ]:
surface_df.info()

In [ ]:
ground_df.info()

In [ ]:
# Analysis post Berlin Wall Fall
# surface_df = surface_df[surface_df["DateTime"] >= "1998-01-01"]

In [ ]:
ground_df = ground_df[ground_df["DateTime"] >= "1998-01-01"]

# Variables Comparison

## Conductivity vs Flow River Rate

In [ ]:
columns = ['Conductivity (µS/cm)', 'Flow River Rate (m³/s)']

In [ ]:
for station_id in surface_df['Station'].unique():
    station_df = surface_df[surface_df['Station'] == station_id][['DateTime'] + columns].copy()
    station_df.dropna(inplace=True)
    
    # Granger Causality Test
    max_lag = 6
    
    for column in columns:
        scaler = MinMaxScaler()
        station_df[column] = scaler.fit_transform(station_df[[column]])
        
    
    
    cond = station_df['Conductivity (µS/cm)']
    flow = station_df['Flow River Rate (m³/s)']
    
    print()
    print('Pearson Correlation')
    print(pearsonr(cond, flow))
    
    # first check for stationarity
    
    # Augmented Dickey-Fuller test
    adf_cond = adfuller(cond)
    adf_flow = adfuller(flow)
    
    print(f"Station {station_id}")
    print("Augmented Dickey-Fuller Test")
    if adf_cond[1] > 0.05 or adf_flow[1] > 0.05:
        print("Non-stationary data")
        print("Differencing data")
        cond = np.diff(cond, n=1)
        flow = np.diff(flow, n=1)
    
    print("Granger Causality Test - Conductivity -> Flow")
    df = pd.DataFrame({
        'cond': cond,
        'flow': flow
    })
    
    grangercausalitytests(df, max_lag, verbose=True)
    
    print()
    print("Granger Causality Test - Flow -> Conductivity")
    df = pd.DataFrame({
        'flow': flow,
        'cond': cond
    })
    
    grangercausalitytests(df, max_lag, verbose=True)
    
    
    fig = go.Figure()
    
    for column in columns:
        
        fig.add_trace(
            go.Scatter(
                x=station_df['DateTime'],
                y=station_df[column],
                mode='lines',
                name=column
            )
        )
        
    fig.update_layout(
        title=f"Surface Station {station_id}",
        xaxis_title="Date",
        yaxis_title="Value"
    )
    
    fig.show()

# Trend Analysis

In [ ]:
diff_columns = ["DateTime", "Station"]
bacteria_columns = [
    "E.Coli (MPN/100ml)",
    "Enterococcus (MPN/100ml)",
    "Coliform (MPN/100ml)"
]

## Surface

### Statistical Analysis

#### Statistical Tests on Stationarity and Trend

In [ ]:
# create dataframe to store the adf and mann-kendall test results for each station

statistics_df = pd.DataFrame(
    index=surface_df.columns.difference(diff_columns + bacteria_columns),
    columns=pd.MultiIndex.from_product([surface_df['Station'].unique().tolist(), ['ADF p-value', 'ADF result', 'MK p-value', 'MK result', 'Slope', 'Slope p-value', 'Trend Percentage']])
)

In [ ]:
for station_id in surface_df["Station"].unique():
    station_df = surface_df[surface_df["Station"] == station_id].copy()
    
    # analysis post Berlin Wall Fall
    station_df = station_df[station_df["DateTime"] >= "1992-01-01"]

    for column in station_df.columns.difference(diff_columns):
        df = station_df[["DateTime", column]].copy()

        df.set_index("DateTime", inplace=True)

        df.dropna(inplace=True)

        date_range = df.index
        date_range = date_range.min(), date_range.max()

        # make sure that the dataframe starts and finishes in the same month
        start_index = df[df.index.month == date_range[1].month].index[0]

        # Slice the dataframe to start from the found index
        df = df.loc[start_index:]

        # ===== Prophet =====

        df.index.name = "ds"

        df = df.reset_index()

        df.rename(columns={column: "y"}, inplace=True)

        # using prophet

        model = Prophet()
        model.fit(df)
        # Make predictions for both columns
        future = model.make_future_dataframe(periods=0)
        forecast = model.predict(future)

        # Merging forecasted data with your original data
        forecasting_final = pd.merge(
            forecast,
            df,
            how="inner",
            on="ds",
        )

        # compute linear regression on trend
        X = np.arange(df.shape[0])
        X = sm.add_constant(X)
        y = df["y"].copy()

        model = sm.OLS(y, X)
        results = model.fit()

        # plot the line of the linear regression
        line = pd.Series(results.predict(X), index=df['ds'])

        fig = go.Figure()

        fig.add_trace(
            go.Scatter(
                x=df['ds'],
                y=df["y"],
                mode="lines",
                name="Original",
            )
        )

        fig.add_trace(
            go.Scatter(
                x=forecasting_final["ds"],
                y=forecasting_final["trend"],
                mode="lines",
                name="Trend",
            )
        )    
        
        # perfrom Augmented Dickey-Fuller test
        adf_result = adfuller(df["y"], autolag="AIC")
        # perform KPSS test
        kpss_result = kpss(df["y"])
        
        # perfrom Mann-Kendall test        
        mk_result = mk.original_test(df["y"] - forecasting_final['yearly'])
        
        print()
        print(f"{column} - Augmented Dickey-Fuller Test")
        print(f"ADF P-value: {adf_result[1]:.4f}")
        print(f"Lag used: {adf_result[2]}")
        if adf_result[1] > 0.05:
            print("Unit root present, data is non-stationary")
        print()
        
        print(f"{column} - KPSS Test")
        print(f"KPSS P-value: {kpss_result[1]:.4f}")
        if kpss_result[1] < 0.05:
            print("Unit root present, data is non-stationary")
        print()
        
        if (adf_result[1] > 0.05 and kpss_result[1] < 0.05) or (adf_result[1] < 0.05 and kpss_result[1] > 0.05):
            print("=== Consistency between tests! ===")
            print()
        
        print(f"{column} - Mann-Kendall Test")
        print(f"Monotonic Trend: {mk_result.trend}")
        print(f"p-value: {mk_result.p:.4f}")
        print()
        slope = results.params.iloc[1]
        print(f"{column} - Slope: {slope}")

        p_value = results.pvalues.iloc[1]
        print(f"{column} - P-value: {p_value}")
        
        statistics_df.loc[column, (station_id, 'ADF p-value')] = adf_result[1]
        statistics_df.loc[column, (station_id, 'ADF result')] = 'Stationary' if adf_result[1] < 0.05 else 'Non-Stationary'
        
        statistics_df.loc[column, (station_id, 'MK p-value')] = mk_result.p
        statistics_df.loc[column, (station_id, 'MK result')] = mk_result.trend
        
        # store the slope
        statistics_df.loc[column, (station_id, 'Slope')] = slope
        statistics_df.loc[column, (station_id, 'Slope p-value')] = p_value
        
        # calculate the percentage of the trend
        trend_percentage = slope / df['y'].mean() * 100
        
        statistics_df.loc[column, (station_id, 'Trend Percentage')] = trend_percentage

        fig.add_trace(
            go.Scatter(
                x=line.index,
                y=line,
                mode="lines",
                name=f"Linear Regression",
                line=dict(dash="dash", color="black"),
            ),
        )

        start_date = df['ds'].min()
        end_date = df['ds'].max()

        fig.update_layout(
            xaxis_title="Date",
            yaxis_title=column,
            font=dict(
                size=18,
            ),
            title=f"{station_id} - {column} - {start_date.strftime('%Y-%m-%d')} - {end_date.strftime('%Y-%m-%d')} - Trend Percentage: {trend_percentage:.2f}% - Slope: {slope:.4f}",
        )

        fig.show()

In [ ]:
# Compute the percentage change for each subsequent year
for station_id in surface_df["Station"].unique():
    station_df = surface_df[surface_df["Station"] == station_id].copy()
    
    # analysis post Berlin Wall Fall
    station_df = station_df[station_df["DateTime"] >= "1992-01-01"]

    for column in station_df.columns.difference(diff_columns):
        df = station_df[["DateTime", column]].copy()

        df.set_index("DateTime", inplace=True)

        df.dropna(inplace=True)
    
        # compute the percentage change with respect to the first complete year
        complete_years = df.groupby(df.index.year).filter(lambda x: len(x) == 12)

        # Get the first complete year (with measurements for all 12 months)
        first_year = complete_years.index.year.min()
        
        last_year = complete_years.index.year.max()
        
        first_year_df = complete_years[complete_years.index.year == first_year]
        
        last_year_df = complete_years[complete_years.index.year == last_year]
        
        percentage_change = (last_year_df[column].mean() - first_year_df[column].mean()) / first_year_df[column].mean() * 100
        
        print(f"{station_id} - {column} - Percentage Change: {percentage_change:.2f}% - First Year: {first_year} - Last Year: {last_year}")
    
        
        

In [ ]:
statistics_df.to_excel(os.path.join(surface_plot_folder, 'trends', "statistics.xlsx"))

#### ACF and PACF

In [ ]:
for station_id in surface_df['Station'].unique():
    station_df = surface_df[surface_df['Station'] == station_id].copy()
    
    for column in station_df.columns.difference(['DateTime', 'Station']):
        
        print(f"{station_id} - {column}")
        plot_acf(station_df[column], lags=20)
        
        
        plot_pacf(station_df[column], lags=20)
        
        plt.show()

#### Causality and Cointegration

"If two or more time-series are cointegrated, then there must be Granger causality between them - either one-way or in both directions. However, the converse is not true."

"So, if your data are cointegrated but you don't find any evidence of causality, you have a conflict in your results. (This might occur if your sample size is too small to satisfy the asymptotics that the cointegration and causality tests rely on.) If you have cointegration and find one-way causality, everything is fine. (You may still be wrong about there being no causality in the other direction.) If your data are not cointegrated, then you have no cross-check on your causality results."

In [ ]:
# Function to perform Johansen test
def johansen_test(data, det_order, k_ar_diff=1):
    """
    Performs the Johansen cointegration test and prints the results.
    
    Parameters:
    data (numpy.ndarray): The time series data for the cointegration test.
    det_order (int): The order of the deterministic terms.
                     -1: No constant or trend.
                      0: Constant term only.
                      1: Constant and trend terms.
    k_ar_diff (int): The number of lags to include in the VAR model.
    """
    result = coint_johansen(data, det_order, k_ar_diff)
    
    print(f'Johansen Test Results (det_order={det_order})')
    print('Trace Statistics:', result.trace_stat)
    print('Critical Values (Trace):', result.trace_stat_crit_vals)
    print()

In [ ]:
def check_stationarity(df):
    """
    Check and make the time series stationary if required.
    """
    for column in df.columns:
        adf = adfuller(df[column])
        kp = kpss(df[column])
        if adf[1] > 0.05 and kp[1] < 0.05:
            print(f'{column} is non-stationary. Differencing the data.')
            df[column] = df[column].diff().dropna()
    return df.dropna()

In [ ]:
coint_dfs = {}

for station_id in surface_df['Station'].unique():
    station_df = surface_df[surface_df['Station'] == station_id].copy()
    
    station_df = station_df.drop(columns=diff_columns + bacteria_columns).dropna()
    
    print(f"Station {station_id}")
    print()
    
    df = check_stationarity(station_df)
    
    coint_df = pd.DataFrame(columns=df.columns, index=df.columns)
    
    # perfrom Engle-Granger test
    for column in df.columns:
        for column2 in df.columns:
            if column == column2:
                continue
            result = coint(df[column], df[column2])
            
            if result[1] < 0.05:
                coint_df.loc[column, column2] = True
            else:
                coint_df.loc[column, column2] = False
                
    coint_dfs[station_id] = coint_df

In [ ]:
coint_df

# the row is cointrated with the column

In [ ]:
%%script false --no-raise-error
for station_id in surface_df['Station'].unique():
    station_df = surface_df[surface_df['Station'] == station_id].copy()
    
    station_df = station_df.drop(columns=diff_columns + bacteria_columns).dropna()
    
    station_df = check_stationarity(station_df)
    
    aic, bic, fpe, hqic = [], [], [], []
    model = VAR(station_df)
    if station_id == 105:
        p = range(0, 7)
    else:
        p = range(0, 17)
    for i in p:
        result = model.fit(i)
        aic.append(result.aic)
        bic.append(result.bic)
        fpe.append(result.fpe)
        hqic.append(result.hqic)
    lags_metrics_df = pd.DataFrame({'AIC': aic, 
                                    'BIC': bic, 
                                    'HQIC': hqic,
                                    'FPE': fpe}, 
                                index=p)    
    fig, ax = plt.subplots(1, 4, figsize=(15, 3), sharex=True)
    lags_metrics_df.plot(subplots=True, ax=ax, marker='o')
    
    # set the title
    plt.suptitle(f"Station {station_id}")
    
    plt.tight_layout()
    

In [ ]:
# select max lag = 1 for all stations

In [ ]:
# multivariate case

def check_stationarity(dataframe):
    """
    Check and make the time series stationary if required.
    """
    
    # use mann-kendall test
    result = mk.original_test(dataframe)
    if result.trend == 'no trend':
        print(f'{column} has no trend. Differencing the data.')
        dataframe = dataframe.diff().dropna()
        dataframe = check_stationarity(dataframe)
    return dataframe.dropna()

def grangers_causation_matrix_multivariate(data, maxlag=12, test='ssr_chi2test', verbose=False):
    """
    Check Granger Causality in a multivariate setting.
    """
    variables = data.columns
    df = pd.DataFrame(np.zeros((len(variables), len(variables))), columns=variables, index=variables)
        
    # Ensure data is stationary
    for column in data.columns:
        data_column = check_stationarity(data[[column]])
        data[column] = data_column
    
    # Select the optimal lag length for the VAR model
    
    # Fit the VAR model
    model = VAR(data)
    model_fitted = model.fit(maxlags=maxlag)
    
    for r in variables:
        for c in variables:
            if c != r:
                test_result = model_fitted.test_causality(c, r, kind=test, verbose=verbose)
                p_value = round(test_result.pvalue, 4)
                df.loc[r, c] = p_value
                if verbose:
                    print(f'Y = {r}, X = {c}, P-Value = {p_value}')
            else:
                df.loc[r, c] = 1
                
    
    
    df.columns = [var + '_x' for var in variables]
    df.index = [var + '_y' for var in variables]
    
    return df, model_fitted.summary(), model_fitted.irf(12)


In [ ]:
causality_matrices = {}

for station_id in surface_df['Station'].unique():
    station_df = surface_df[surface_df['Station'] == station_id].copy()
    
    causality_matrix, summary, irf = grangers_causation_matrix_multivariate(station_df.drop(columns=diff_columns + bacteria_columns).dropna(), maxlag=1)
    
    causality_matrices[station_id] = causality_matrix
    
    print(f"Station {station_id}")
    print(summary)
    
    ax = irf.plot(orth=True)
    ax.set_size_inches(40, 20)
    
    plt.show()
    

In [ ]:
# check if the cointration matrix is consistent with the Granger Causality matrix
# if the cointration matrix is True, then the Granger Causality matrix should have a p-value < 0.05

for station_id in surface_df['Station'].unique():
    
    coint_df = coint_dfs[station_id]
    causality_matrix = causality_matrices[station_id]
    
    print(f"Station {station_id}")
    print()
    
    count = 0
    cons_count = 0
    
    for column in coint_df.columns:
        for column2 in coint_df.columns:
            
            if column == column2:
                continue
            
            count += 1
            
            if coint_df.loc[column, column2]:
                if causality_matrix.loc[column + '_y', column2 + '_x'] < 0.05:
                    print(f"{column} -> {column2} is consistent")
                    cons_count += 1
                else:
                    print(f"{column} -> {column2} is not consistent")
            else:
                if causality_matrix.loc[column + '_y', column2 + '_x'] < 0.05:
                    print(f"{column} -> {column2} is not consistent")
                else:
                    print(f"{column} -> {column2} is consistent")
                    cons_count += 1
                    
    print()
    print()
    
    print(f"Consistency: {cons_count}/{count}")

In [ ]:
for station_id, causality_matrix in causality_matrices.items():
    
    fig = go.Figure()
    
    fig.add_trace(
        go.Heatmap(
            z=causality_matrix.values,
            x=causality_matrix.columns,
            y=causality_matrix.index,
        )
    )
    
    annotations = []
    for i in range(causality_matrix.shape[0]):
        for j in range(causality_matrix.shape[1]):
            
            if i == j:
                continue
            
            value = causality_matrix.iloc[i, j]
            if value < 0.05:
                annotations.append(
                    dict(
                        x=causality_matrix.columns[j],
                        y=causality_matrix.index[i],
                        text=f"{value:.2f}",
                        showarrow=False,
                        font=dict(color="white"),
                        xref="x",
                        yref="y"
                    )
                )
    
    fig.update_layout(
        title=f"Granger Causality p-value Matrix - Station {station_id}",
        xaxis_title="X",
        yaxis=dict(
            title="Y",
            autorange='reversed'  # Reverse the order of the y-axis
        ),
        annotations= annotations + [
                dict(
                    xref='paper',
                    yref='paper',
                    x=-0.2,
                    y=-0.6,
                    showarrow=False,
                    text='Note: if a given p-value is < significance level (0.05), then, <br> the corresponding X series (column) causes the Y (row).',
                    font=dict(
                        size=12
                    )
                )
            ]
    )
    
    fig.show()

In [ ]:
# store the results in an excel file
for station_id, causality_matrix in causality_matrices.items():
    causality_matrix.to_excel(os.path.join(surface_plot_folder, 'granger', f"{station_id}.xlsx"))

##### Granger Causality Graph

In [ ]:

for station_id in surface_df['Station'].unique():
    matrix = causality_matrices[station_id]

    # remove the _x and _y from the column names
    matrix.columns = matrix.columns.str.replace('_x', '')
    matrix.index = matrix.index.str.replace('_y', '')

    # Create a directed graph
    G = nx.DiGraph()

    # Add nodes for all effects (rows) and causes (columns)
    G.add_nodes_from(matrix.columns, bipartite=0)  # Causes
    G.add_nodes_from(matrix.index, bipartite=1)    # Effects

    # Add edges for significant Granger causality (p-value < 0.05)
    threshold = 0.05
    for cause in matrix.columns:
        for effect in matrix.index:
            if matrix.loc[effect, cause] < threshold:
                G.add_edge(cause, effect)

    plt.figure(figsize=(12, 6))
    pos = nx.circular_layout(G)
    nx.draw(G, pos, with_labels=True, node_color='skyblue', node_size=3000, edge_color='black',
            arrows=True, arrowsize=20, font_size=12, font_color='darkblue')
    plt.title(f"Granger Causality Graph - Station {station_id}")

    # make the plot wider
    plt.savefig(
        os.path.join(paper_plots_folder, 'Berlin', f"granger_causality_{station_id}.png"),
        bbox_inches='tight',
        dpi=600
    )

In [ ]:
# univariate case

def select_best_lag(data, maxlag):
    """
    Select the best lag length using information criteria.
    """
    model = VAR(data)
    lag_order = model.select_order()
    return lag_order.aic


# Check for stationarity using both adfuller and kpss tests
def check_stationarity(series):
    # use mann-kendall test
    result = mk.original_test(series)
    if result.p < 0.05:
        return True
    return False

def grangers_causation_matrix(data, variables, maxlag=12, test='ssr_chi2test', verbose=False):
    df = pd.DataFrame(np.zeros((len(variables), len(variables))), columns=variables, index=variables)
    
    df = df.astype(object)
    
    lag_df = pd.DataFrame(np.zeros((len(variables), len(variables))), columns=variables, index=variables)
    
    for c in df.columns:
        for r in df.index:
            if c != r:
                granger_df = data[[c, r]]
                if data[c].isna().any() or data[r].isna().any():
                    granger_df = granger_df[[r, c]].dropna()
                
                try:
                    adfuller(granger_df[c])
                    adfuller(granger_df[r])
                except Exception as e:
                    print(e)
                    continue
                
                if not check_stationarity(granger_df[c]):
                    print(f"{c} is non-stationary")
                    print("Differencing data")
                    granger_df[c] = granger_df[c].diff().dropna()
                
                if not check_stationarity(granger_df[r]):
                    print(f"{r} is non-stationary")
                    print("Differencing data")
                    granger_df[r] = granger_df[r].diff().dropna()
                    
                granger_df.dropna(inplace=True)
                
                # Select the best lag
                # selected_lag = select_best_lag(granger_df, maxlag)
                
                test_result = grangercausalitytests(granger_df, maxlag=10, verbose=False)
                
                # p_values = [round(test_result[i+1][0][test][1], 4) for i in range(selected_lag)]
                
                # the lag is the index of the list + 1
                # store all the p-values to make a plot later
                p_values = [round(test_result[i+1][0][test][1], 4) for i in range(10)]
                statistics = [test_result[i+1][0][test][0] for i in range(10)]
                
                # need to store as list to avoid the error
                
                # store both the p values and the statistics in the same cell of df
                df.loc[r, c] = [p_values, statistics]
                
            else:
                df.loc[r, c] = 1
                
    df.columns = [var + '_x' for var in variables]
    df.index = [var + '_y' for var in variables]
    
    return df, lag_df


In [ ]:
causality_matrices = {}
lag_matrices = {}

for station_id in surface_df['Station'].unique():
    station_df = surface_df[surface_df['Station'] == station_id].copy()
    
    causality_matrix, lag_matrix = grangers_causation_matrix(station_df, variables=station_df.columns.difference(diff_columns + bacteria_columns), maxlag=12)
    
    causality_matrices[station_id] = causality_matrix
    lag_matrices[station_id] = lag_matrix

In [ ]:
causality_matrices[station_id]

In [ ]:
for station_id, causality_matrix in causality_matrices.items():
    
    for i in range(causality_matrix.shape[0]):
        for j in range(causality_matrix.shape[1]):
            if i != j:
                test_result = causality_matrix.iloc[i, j]
                
                # check if the test result is a list
                if not isinstance(test_result, list):
                    continue
                
                p_values = test_result[0]
                statistics = test_result[1]
                
                lags = np.arange(1, len(p_values) + 1)
                
                fig = go.Figure()
                
                fig.add_trace(
                    go.Scatter(
                        x=lags,
                        y=p_values,
                        mode='lines',
                        name='P-Value'
                    )
                )
                
                # fig.add_trace(
                #     go.Scatter(
                #         x=lags,
                #         y=statistics,
                #         mode='lines',
                #         name='F-Statistic'
                #     )
                # )
                
                fig.update_layout(
                    title=f"Station {station_id} - Granger Causality Test - {causality_matrix.columns[j]} -> {causality_matrix.index[i]}",
                    xaxis_title="Lag",
                    yaxis_title="Value"
                )
                
                fig.show()              

In [ ]:
for station_id, lag_matrix in lag_matrices.items():
    
    px.imshow(
        lag_matrix,
        labels=dict(x="X", y="Y", color="Lag"),
        title=f"Lag Matrix - Station {station_id}",
        text_auto=True,
        color_continuous_scale='Viridis',
        width=800,
        height=800
    ).show()

### Data Analysis

#### TS Decomposition

In [ ]:
color = 'rgb(200, 2, 110)'

In [ ]:
for station_id in surface_df['Station'].unique():
    station_df = surface_df[surface_df['Station'] == station_id].copy()
    
    station_df.set_index('DateTime', inplace=True)
    
    station_df = station_df.drop(columns=bacteria_columns)
    
    # Analysis post Berlin Wall Fall
    station_df = station_df[station_df.index >= "1992-01-01"]
    
    for column in station_df.columns.difference(diff_columns):
        
        df = pd.DataFrame({
            'ds': station_df.index,
            'y': station_df[column]
        })
        
        model = Prophet(weekly_seasonality=False, daily_seasonality=False)
        model.fit(df)
        # Make predictions for both columns
        future = model.make_future_dataframe(periods=0)
        forecast = model.predict(future)
        
        
        # compute linear regression on trend
        X = np.arange(df.dropna().shape[0])
        X = sm.add_constant(X)
        y = df["y"].dropna().copy()
        
        model = sm.OLS(y, X)
        results = model.fit()
        
        # get the slope
        slope = results.params.iloc[1]
        
        # compute the decade increase or decrease
        variation = slope * 120
        
        fig = go.Figure()

        fig.add_trace(
            go.Scatter(
                x=station_df.index,
                y=station_df[column],
                mode='lines',
                name='Historical',
                line=dict(
                    color='black',
                    width=0.6
                )
            )
        )
        
        fig.add_trace(
            go.Scatter(
                x=forecast['ds'],
                y=forecast['trend'],
                mode='lines',
                name='Trend',
                line=dict(color=color)
            )
        )
        
        # get the unit of measure for the column
        uom = column.split('(')[-1].replace(')', '')
        
        # add a box with the variation with the plus or minus sign
        fig.add_annotation(
            go.layout.Annotation(
                x=0.05,
                y=0.95,
                xref='paper',
                yref='paper',
                text=f"Decade Variation: +{variation:.2f} {uom}" if variation > 0 else f"Decade Variation: {variation:.2f} {uom}",
                showarrow=False,
                font=dict(
                    size=12,
                    color='red'
                )
            )
        )
        
        fig.update_layout(
            xaxis_title="Time",
            yaxis_title=f"{column}"
        )
        
        if not os.path.exists(os.path.join(surface_plot_folder, 'trends', f"station_{station_id}")):
            os.makedirs(os.path.join(surface_plot_folder, 'trends', f"station_{station_id}")) 
            
        column_ = column.replace("/", "_")
        
        # fig.write_image(
        #     os.path.join(
        #         surface_plot_folder,
        #         'trends',
        #         f"station_{station_id}",
        #         f"{column_}.png"
        #     ),
        #     scale=3
        # )
        
        fig.show()

##### Monthly

In [ ]:
# Perform trend analysis for each month

# Build interaction plot

months = surface_df['DateTime'].dt.month.unique()
months.sort()

# Get the names of the months
months_name = [pd.to_datetime(f"{month}-01-2021").strftime("%B") for month in months]

# Create dataframe to store the adf and mann-kendall test results for each station for each month
statistics_df = pd.DataFrame(
    index=pd.MultiIndex.from_product([surface_df.columns.difference(diff_columns + bacteria_columns), months_name]),
    columns=pd.MultiIndex.from_product([surface_df['Station'].unique().tolist(), ['ADF p-value', 'ADF result', 'MK p-value', 'MK result', 'Slope', 'Slope p-value', 'Trend Percentage']])
)

legend_added = set()

# Calculate global min and max for each variable
global_min_max = {}
for column in surface_df.columns.difference(diff_columns + bacteria_columns):
    global_min_max[column] = (surface_df[column].min(), surface_df[column].max())

for station_id in surface_df['Station'].unique():
    
    station_df = surface_df[surface_df['Station'] == station_id].copy()
    
    station_df.set_index('DateTime', inplace=True)
    
    station_df = station_df.drop(columns=bacteria_columns)
    
    # Analysis post Berlin Wall Fall
    station_df = station_df[station_df.index >= "1992-01-01"]
    
    for column in station_df.columns.difference(diff_columns):
        
        fig = make_subplots(
            4,
            3,
            subplot_titles=months_name,
            vertical_spacing=0.08,  # Reduce vertical spacing
            horizontal_spacing=0.05  # Reduce horizontal spacing
        )
        
        for i, month in enumerate(months):
            
            month_df = station_df[station_df.index.month == month]
                       
            # Perform Augmented Dickey-Fuller test
            try:
                adf_result = adfuller(month_df[column].dropna(), autolag="AIC")
            except Exception as e:
                print(e)
                continue
            
            # Perform Mann-Kendall test
            mk_result = mk.original_test(month_df[column])
            
            # Store the results in the statistics dataframe
            statistics_df.loc[(column, months_name[i]), (station_id, 'ADF p-value')] = adf_result[1]
            statistics_df.loc[(column, months_name[i]), (station_id, 'ADF result')] = 'Stationary' if adf_result[1] < 0.05 else 'Non-Stationary'
            
            statistics_df.loc[(column, months_name[i]), (station_id, 'MK p-value')] = mk_result.p
            statistics_df.loc[(column, months_name[i]), (station_id, 'MK result')] = mk_result.trend
            
            df = pd.DataFrame({
                'ds': month_df.index,
                'y': month_df[column]
            })
            
            model = Prophet(yearly_seasonality=False, weekly_seasonality=False, daily_seasonality=False)
            model.fit(df)
            # Make predictions for both columns
            future = model.make_future_dataframe(periods=0)
            forecast = model.predict(future)
            
            # compute linear regression on trend
            X = np.arange(df.shape[0])
            X = sm.add_constant(X)
            y = df["y"].copy()

            model = sm.OLS(y, X)
            results = model.fit()
            
            slope = results.params.iloc[1]

            p_value = results.pvalues.iloc[1]
            
            # store the slope
            statistics_df.loc[(column, months_name[i]), (station_id, 'Slope')] = slope
            statistics_df.loc[(column, months_name[i]), (station_id, 'Slope p-value')] = p_value
            
            # calculate the percentage of the trend
            first_value = forecast['trend'].iloc[0]
            last_value = forecast['trend'].iloc[-1]
            
            trend_percentage = (last_value - first_value) / first_value * 100
            
            statistics_df.loc[(column, months_name[i]), (station_id, 'Trend Percentage')] = trend_percentage
                
            fig.add_trace(
                go.Scatter(
                    x=month_df.index,
                    y=month_df[column],
                    mode='lines',
                    name='Historical',
                    line=dict(
                        color='black',
                        width=0.6
                    ),
                    showlegend=True if column not in legend_added else False
                ),
                row=(i // 3) + 1,
                col=(i % 3) + 1
            )
            
            fig.add_trace(
                go.Scatter(
                    x=forecast['ds'],
                    y=forecast['trend'],
                    mode='lines',
                    name='Trend',
                    line=dict(color='blue'),
                    showlegend=True if column not in legend_added else False
                ),
                row=(i // 3) + 1,
                col=(i % 3) + 1
            )
            
            legend_added.add(column)
            
        # Set the same y-axis range for all subplots
        fig.update_yaxes(range=global_min_max[column])
        
        fig.add_annotation(dict(
            x=0.5,
            y=-0.08,
            showarrow=False,
            text="Time",
            xref="paper",
            yref="paper",
            font=dict(size=20)
        ))

        fig.add_annotation(dict(
            x=-0.08,
            y=0.5,
            showarrow=False,
            text=column,
            textangle=-90,
            xref="paper",
            yref="paper",
            font=dict(size=20)
        )) 
            
        fig.update_layout(
            height=1000,
            width=1200,
            title={
                'text': f"Station {station_id}",
                'y': 0.98,  # Vertical position
                'x': 0.5,  # Horizontal position
                'xanchor': 'center',
                'yanchor': 'top'
            },
            font=dict(size=18),
            margin=dict(t=80, l=80, b=100, r=40)  # Adjust margins
        )

        if not os.path.exists(os.path.join(surface_plot_folder, 'trends', 'monthwise', f"station_{station_id}")):
            os.makedirs(os.path.join(surface_plot_folder, 'trends', 'monthwise', f"station_{station_id}")) 
            
        column_ = column.replace("/", "_")
        
        fig.write_image(
            os.path.join(
                surface_plot_folder,
                'trends',
                'monthwise',
                f"station_{station_id}",
                f"{column_}.png"
            ),
            scale=3
        )

In [ ]:
statistics_df.to_excel(os.path.join(surface_plot_folder, 'trends', 'monthwise', "statistics.xlsx"))

#### Correlation

##### Cross-Correlation (lags)

In [ ]:
def check_stationarity(df):
    """
    Check and make the time series stationary if required.
    """
    for column in df.columns:
        adf = adfuller(df[column])
        kp = kpss(df[column])
        if adf[1] > 0.05 and kp[1] < 0.05:
            print(f'{column} is non-stationary. Differencing the data.')
            df[column] = df[column].diff().dropna()
    return df.dropna()

In [ ]:
def compute_cross_corr(ts1, ts2, max_lag):
    lags = range(-max_lag, max_lag + 1)
    cross_corr = [ts1.corr(ts2.shift(lag)) for lag in lags]
    return lags, cross_corr

In [ ]:
# check if a time series is stationary, if not, difference it

correlation_results_df = pd.DataFrame(
    index=surface_df.columns.difference(diff_columns + bacteria_columns),
    columns=pd.MultiIndex.from_product([surface_df['Station'].unique().tolist(), surface_df.columns.difference(diff_columns + bacteria_columns)])
)

for station_id in surface_df['Station'].unique():
    station_df = surface_df[surface_df['Station'] == station_id].copy()
    
    station_df = station_df.drop(columns=diff_columns + bacteria_columns).dropna()
    
    station_df = check_stationarity(station_df)
    
    # normalize the data
    for column in station_df.columns:
        scaler = MinMaxScaler()
        station_df[column] = scaler.fit_transform(station_df[[column]])
        
    
    max_lag = 12
    
    for column in station_df.columns:
        for column2 in station_df.columns:
            if column == column2:
                continue
            
            lags, cross_corr = compute_cross_corr(station_df[column], station_df[column2], max_lag)
            
            fig = go.Figure()
            
            fig.add_trace(
                go.Scatter(
                    x=list(lags),
                    y=cross_corr,
                    mode='lines+markers',
                    name='Cross Correlation'
                )
            )
            
            fig.update_layout(
                title=f"Station {station_id} - {column} vs {column2}",
                xaxis_title="Lag (Months)",
                yaxis_title="Cross Correlation"
            )
            
            if not os.path.exists(os.path.join(surface_cross_corr_folder, f"station_{station_id}")):
                os.makedirs(os.path.join(surface_cross_corr_folder, f"station_{station_id}"))
                
                
            column_ = column.replace("/", "_")
            column2_ = column2.replace("/", "_")
                
            fig.write_image(
                os.path.join(
                    surface_cross_corr_folder,
                    f"station_{station_id}",
                    f"{column_}_vs_{column2_}.png"
                ),
                scale=3
            )
            
            corr, pvalue = pearsonr(station_df[column], station_df[column2])
            
            # put the correlation just if the p-value is less than 0.05 and in the upper triangle
            if pvalue < 0.05:
                correlation_results_df.loc[column, (station_id, column2)] = corr
            else:
                correlation_results_df.loc[column, (station_id, column2)] = np.nan
                
            

In [ ]:
correlation_results_df.to_excel(os.path.join(surface_cross_corr_folder, "correlation_results.xlsx"))

##### Year by Year Correlation

In [ ]:
# perform year by year correlation
correlation_results = pd.DataFrame(
    index=pd.MultiIndex.from_product([surface_df.columns.difference(diff_columns + bacteria_columns), sorted(surface_df['DateTime'].dt.year.unique())]),
    columns=pd.MultiIndex.from_product([surface_df['Station'].unique().tolist(), surface_df.columns.difference(diff_columns + bacteria_columns)])
)

for station_id in surface_df['Station'].unique():
    station_df = surface_df[surface_df['Station'] == station_id].copy()
    
    if station_id == 305:
            # remove flow river
            station_df.drop(
                columns=['Flow River Rate (m³/s)'],
                inplace=True
            )
    
    station_df = station_df.drop(columns=bacteria_columns).dropna()
    
    # normalize the data
    for column in station_df.columns.difference(diff_columns):
        scaler = MinMaxScaler()
        station_df[column] = scaler.fit_transform(station_df[[column]])
    
    correlation_results[station_id] = {}
    
    for year in station_df['DateTime'].dt.year.unique():
        
        year_df = station_df[station_df['DateTime'].dt.year == year]
        
        year_df.drop(columns=diff_columns, inplace=True)
        
        for column in year_df.columns:
            for column2 in year_df.columns:
                if column == column2:
                    continue
                
                result = pearsonr(year_df[column], year_df[column2])
                
                correlation_results.loc[(column, year), (station_id, column2)] = result

In [ ]:
# plot the correlation results

for station_id in surface_df['Station'].unique():
        
        station_df = surface_df[surface_df['Station'] == station_id].copy()
        
        station_df = station_df.drop(columns=bacteria_columns).dropna()
        
        station_df.set_index('DateTime', inplace=True)
        
        for column in station_df.columns.difference(diff_columns):
            
            fig = go.Figure()
            
            for column2 in station_df.columns.difference(diff_columns):
                
                if column == column2:
                    continue
                
                years = correlation_results.loc[column, (station_id, column2)].index
                
                correlation = correlation_results.loc[(column, years), (station_id, column2)].dropna()            
                
                fig.add_trace(
                    go.Scatter(
                        x=correlation.index.get_level_values(1),
                        y=correlation.apply(lambda x: x[0]),
                        mode='lines+markers',
                        name=column2
                    )
                )
                
            fig.update_layout(
                title=f"Station {station_id} - {column} vs Other Parameters",
                xaxis_title="Year",
                yaxis_title="Pearson Correlation Coefficient"
            )
            
            
            if not os.path.exists(os.path.join(surface_corr_folder, f"station_{station_id}")):
                os.makedirs(os.path.join(surface_corr_folder, f"station_{station_id}"))
                
            column_ = column.replace("/", "_")
            
            fig.update_yaxes(range=[-1, 1])
            
            # add a horizontal line at 0
            fig.add_shape(
                type="line",
                x0=years.min(),
                y0=0,
                x1=years.max(),
                y1=0,
                line=dict(
                    color="black",
                    width=1,
                    dash="dashdot"
                )
            )
            
            fig.write_image(
                os.path.join(
                    surface_corr_folder,
                    f"station_{station_id}",
                    f"{column_}.png"
                ),
                scale=3,
                width=1200,
            )
            
            if station_id == 305:
                fig.show()

In [ ]:
correlation_results.to_excel(os.path.join(surface_corr_folder, "correlation_results.xlsx"))

#### Regression for Average Variations

##### Month by Month Variation for each Year

A time series of each year is built, the trend is computed with Prophet and a linear regression on the trend is computed in order to get the slope, which is the average month variation for the given year. 

In [ ]:
# perform trend analysis for each year

# build interaction plot

years = surface_df['DateTime'].dt.year.unique()
years.sort()

# convert the years to int
years = [int(year) for year in years]

# create dataframe to store the adf and mann-kendall test results for each station for each year
slopes_df = pd.DataFrame(
    index=pd.MultiIndex.from_product([surface_df.columns.difference(diff_columns + bacteria_columns), years]),
    columns=pd.MultiIndex.from_product([surface_df['Station'].unique().tolist(), ['Slope', 'P-Value', 'Error', 'Trend Percentage']])
)

legend_added = set()

for station_id in surface_df['Station'].unique():
    
    station_df = surface_df[surface_df['Station'] == station_id].copy()
    
    station_df.set_index('DateTime', inplace=True)
    
    station_df = station_df.drop(columns=bacteria_columns)
    
    # station_df.dropna(inplace=True)
    
    for column in station_df.columns.difference(diff_columns):
        
        # fig = make_subplots(
        #     4,
        #     3,
        #     subplot_titles=years,
        #     vertical_spacing=0.08,  # Reduce vertical spacing
        #     horizontal_spacing=0.05  # Reduce horizontal spacing
        # )
        
        for i, year in enumerate(years):
            
            year_df = station_df[station_df.index.year == year]
            
            # if the dataframe contains only NaN values, skip the year
            if year_df[column].isna().all():
                continue
            
            df = pd.DataFrame({
                'ds': year_df.index,
                'y': year_df[column]
            })
             
            
            model = Prophet(yearly_seasonality=False, weekly_seasonality=False, daily_seasonality=False)
            
            model.fit(df)    
            
            # Make predictions for both columns
            future = model.make_future_dataframe(periods=0)
            forecast = model.predict(future)
            
            # compute the linear regression on the trend
            X = np.arange(forecast.shape[0])
            X = sm.add_constant(X)
            y = forecast['trend'].copy()
            
            model = sm.OLS(y, X)
            results = model.fit()
            
            slope = results.params.iloc[1]
            p_value = results.pvalues.iloc[1]
            st_error = results.bse.iloc[1]
            
            # store the results in the dataframe
            slopes_df.loc[(column, year), (station_id, 'Slope')] = round(slope, 4)
            slopes_df.loc[(column, year), (station_id, 'P-Value')] = round(p_value, 4)
            slopes_df.loc[(column, year), (station_id, 'Error')] = round(st_error, 4)
            
            # compute the percentage of the trend
            trend_percentage = (slope / forecast['trend'].mean()) * 100
            slopes_df.loc[(column, year), (station_id, 'Trend Percentage')] = round(trend_percentage, 4)

In [ ]:
slopes_df.to_excel(os.path.join(surface_plot_folder, 'trends', "slopes.xlsx"))

##### Year by Year Variation for each Month

A time series of each month is built, the trend is computed with Prophet and a linear regression on the trend is computed in order to get the slope, which is the average year variation for the given month. 

In [ ]:
# perform trend analysis for each month

# build interaction plot

months = surface_df['DateTime'].dt.month.unique()
months.sort()

# get the names of the months
months_name = [pd.to_datetime(f"{month}-01-2021").strftime("%B") for month in months]

# create dataframe to store the adf and mann-kendall test results for each station for each month
slopes_df = pd.DataFrame(
    index=pd.MultiIndex.from_product([surface_df.columns.difference(diff_columns + bacteria_columns), months_name]),
    columns=pd.MultiIndex.from_product([surface_df['Station'].unique().tolist(), ['Slope', 'P-Value', 'Error', 'Trend Percentage']])
)

legend_added = set()

for station_id in surface_df['Station'].unique():
    
    station_df = surface_df[surface_df['Station'] == station_id].copy()
    
    station_df.set_index('DateTime', inplace=True)
    
    station_df = station_df.drop(columns=bacteria_columns)
    
    for column in station_df.columns.difference(diff_columns):
        
        # fig = make_subplots(
        #     4,
        #     3,
        #     subplot_titles=months_name,
        #     vertical_spacing=0.08,  # Reduce vertical spacing
        #     horizontal_spacing=0.05  # Reduce horizontal spacing
        # )
        
        for i, month in enumerate(months):
            
            month_df = station_df[station_df.index.month == month]
            
            df = pd.DataFrame({
                'ds': month_df.index,
                'y': month_df[column]
            })
            
            model = Prophet(yearly_seasonality=False, weekly_seasonality=False, daily_seasonality=False)
            model.fit(df)
            # Make predictions for both columns
            future = model.make_future_dataframe(periods=0)
            forecast = model.predict(future)
            
            # compute the linear regression on the trend
            X = np.arange(forecast.shape[0])
            X = sm.add_constant(X)
            y = forecast['trend'].copy()
            
            model = sm.OLS(y, X)
            results = model.fit()
            
            slope = results.params.iloc[1]
            p_value = results.pvalues.iloc[1]
            st_error = results.bse.iloc[1]
            
            # store the results in the dataframe
            slopes_df.loc[(column, months_name[i]), (station_id, 'Slope')] = slope
            slopes_df.loc[(column, months_name[i]), (station_id, 'P-Value')] = p_value
            slopes_df.loc[(column, months_name[i]), (station_id, 'Error')] = st_error
            
            # compute the percentage of the trend
            trend_percentage = (slope / forecast['trend'].mean()) * 100
            slopes_df.loc[(column, months_name[i]), (station_id, 'Trend Percentage')] = trend_percentage
            

In [ ]:
slopes_df.to_excel(os.path.join(surface_plot_folder, 'trends', 'monthwise', "slopes.xlsx"))

In [ ]:
# per il momento lascia qua, potrebbe tornare utile se si decidesse di calcolare una linear regression tra ogni changepoint

for station_id in surface_df['Station'].unique():
    
    station_df = surface_df[surface_df['Station'] == station_id].copy()
    
    station_df.set_index('DateTime', inplace=True)
    
    for column in station_df.columns.difference(diff_columns + bacteria_columns):
        
        df = pd.DataFrame(
            {
                'ds': station_df.index,
                'y': station_df[column]
            }
        )
        
        model = Prophet(weekly_seasonality=False, daily_seasonality=False)
        model.fit(df)
        
        future = model.make_future_dataframe(periods=0)
        forecast = model.predict(future)
        
        # add the changepoints detected by Prophet
        changepoints = model.changepoints
    
        signif_changepoints = changepoints[
            np.abs(np.nanmean(model.params['delta'], axis=0)) >= 0.1
        ]
        
        changepoints_values = forecast.loc[forecast['ds'].isin(signif_changepoints), 'trend']
        
        trend = forecast['trend']
        
        # compute linear regression on trend
        X = np.arange(trend.shape[0])
        X = sm.add_constant(X)
        y = trend.copy()
        
        model = sm.OLS(y, X)
        results = model.fit()
        
        # plot the line of the linear regression
        line = pd.Series(results.predict(X), index=forecast['ds'])
        
        # get the slope with the p-value and the standard error
        slope = results.params.iloc[1]
        p_value = results.pvalues.iloc[1]
        std_err = results.bse.iloc[1]
        
        fig = go.Figure()
        
        fig.add_trace(
            go.Scatter(
                x=station_df.index,
                y=station_df[column],
                mode='lines',
                name='Original',
                line=dict(color='black')
            )
        )
        
        fig.add_trace(
            go.Scatter(
                x=forecast['ds'],
                y=trend,
                mode='lines',
                name='Trend',
                line=dict(color=color)
            )
        )
        
        fig.add_trace(
            go.Scatter(
                x=line.index,
                y=line,
                mode='lines',
                name='Linear Regression',
                line=dict(color='red', dash='dash')
            )
        )
        
        fig.add_trace(
            go.Scatter(
                x=signif_changepoints,
                y=changepoints_values,
                mode='markers',
                name='Changepoints',
                marker=dict(
                    color='green',
                    size=10
                )
            )
        )
        
        fig.update_layout(
            title=f"{column} - Slope: {slope:.3f}, P-Value: {p_value:.3f}, Std. Error: {std_err:.3f}",
            xaxis_title="Date",
            yaxis_title="Value"
        )

        fig.show()
        

#### Pair Comparison same axis

In [ ]:
from itertools import combinations

In [ ]:
feature_columns = surface_df.columns.difference(diff_columns + bacteria_columns)

for station_id in surface_df['Station'].unique():
    station_df = surface_df[surface_df['Station'] == station_id].copy()
    station_df.set_index('DateTime', inplace=True)
    
    for first_col, second_col in combinations(feature_columns, 2):
        
        fig = make_subplots(specs=[[{"secondary_y": True}]])
        
        fig.add_trace(
            go.Scatter(
                x=station_df.index,
                y=station_df[first_col],
                mode='lines',
                name=first_col
            ),
            secondary_y=False
        )
        
        fig.add_trace(
            go.Scatter(
                x=station_df.index,
                y=station_df[second_col],
                mode='lines',
                name=second_col
            ),
            secondary_y=True
        )
        
        fig.update_layout(
            title=f"Station {station_id} - {first_col} vs {second_col}",
            xaxis_title="Date",
            yaxis=dict(
                title=first_col,
                showgrid=False
            ),
            yaxis2=dict(
                title=second_col,
                overlaying='y',
                side='right'
            )
        )
        
        if station_id == 305:
            fig.show()
        

## Ground

### Statistical Analysis

#### Statistical Tests on Stationarity and Trend

In [ ]:
# create dataframe to store the adf and mann-kendall test results for each station

statistics_df = pd.DataFrame(
    index=ground_df.columns.difference(diff_columns),
    columns=pd.MultiIndex.from_product([ground_df['Station'].unique().tolist(), ['ADF p-value', 'ADF result', 'MK p-value', 'MK result', 'Slope', 'Slope p-value']])
)

In [ ]:
for station_id in ground_df["Station"].unique():
    station_df = ground_df[ground_df["Station"] == station_id].copy()

    for column in station_df.columns.difference(diff_columns):
        df = station_df[["DateTime", column]].copy()

        df.set_index("DateTime", inplace=True)

        df.dropna(inplace=True)

        date_range = df.index
        date_range = date_range.min(), date_range.max()

        # make sure that the dataframe starts and finishes in the same month
        start_index = df[df.index.month == date_range[1].month].index[0]

        # Slice the dataframe to start from the found index
        df = df.loc[start_index:]

        # ===== Prophet =====

        df.index.name = "ds"

        df = df.reset_index()

        df.rename(columns={column: "y"}, inplace=True)

        # using prophet

        model = Prophet()
        model.fit(df)
        # Make predictions for both columns
        future = model.make_future_dataframe(periods=0)
        forecast = model.predict(future)

        # Merging forecasted data with your original data
        forecasting_final = pd.merge(
            forecast,
            df,
            how="inner",
            on="ds",
        )

        # compute linear regression on trend
        X = np.arange(df.shape[0])
        X = sm.add_constant(X)
        y = df["y"].copy()

        model = sm.OLS(y, X)
        results = model.fit()

        # plot the line of the linear regression
        line = pd.Series(results.predict(X), index=df['ds'])

        fig = go.Figure()

        fig.add_trace(
            go.Scatter(
                x=df['ds'],
                y=df["y"],
                mode="lines",
                name="Original",
            )
        )

        fig.add_trace(
            go.Scatter(
                x=forecasting_final["ds"],
                y=forecasting_final["trend"],
                mode="lines",
                name="Trend",
            )
        )
        
        # perfrom Augmented Dickey-Fuller test
        adf_result = adfuller(df["y"], autolag="AIC") if df['y'].unique().shape[0] > 1 else (0, 1, 0, 0, 0, 0)
        # perform KPSS test
        kpss_result = kpss(df["y"])
        
        # perfrom Mann-Kendall test        
        mk_result = mk.original_test(df["y"] - forecasting_final['yearly'])
        
        print()
        print(f"{column} - Augmented Dickey-Fuller Test")
        print(f"ADF P-value: {adf_result[1]:.4f}")
        print(f"Lag used: {adf_result[2]}")
        if adf_result[1] > 0.05:
            print("Unit root present, data is non-stationary")
        print()
        
        print(f"{column} - KPSS Test")
        print(f"KPSS P-value: {kpss_result[1]:.4f}")
        if kpss_result[1] < 0.05:
            print("Unit root present, data is non-stationary")
        print()
        
        if (adf_result[1] > 0.05 and kpss_result[1] < 0.05) or (adf_result[1] < 0.05 and kpss_result[1] > 0.05):
            print("=== Consistency between tests! ===")
            print()
        
        print(f"{column} - Mann-Kendall Test")
        print(f"Monotonic Trend: {mk_result.trend}")
        print(f"p-value: {mk_result.p:.4f}")
        print()
        slope = results.params.iloc[1]
        print(f"{column} - Slope: {slope}")

        p_value = results.pvalues.iloc[1]
        print(f"{column} - P-value: {p_value}")
        
        statistics_df.loc[column, (station_id, 'ADF p-value')] = adf_result[1]
        statistics_df.loc[column, (station_id, 'ADF result')] = 'Stationary' if adf_result[1] < 0.05 else 'Non-Stationary'
        
        statistics_df.loc[column, (station_id, 'MK p-value')] = mk_result.p
        statistics_df.loc[column, (station_id, 'MK result')] = mk_result.trend
        
        statistics_df.loc[column, (station_id, 'Slope')] = slope
        statistics_df.loc[column, (station_id, 'Slope p-value')] = p_value

        fig.add_trace(
            go.Scatter(
                x=line.index,
                y=line,
                mode="lines",
                name=f"Linear Regression",
                line=dict(dash="dash", color="black"),
            ),
        )

        start_date = df['ds'].min()
        end_date = df['ds'].max()

        fig.update_layout(
            xaxis_title="Date",
            yaxis_title=column,
            font=dict(
                size=18,
            ),
            title=f"{station_id} - {column} - {start_date.strftime('%Y-%m-%d')} - {end_date.strftime('%Y-%m-%d')}",
        )

        fig.show()

In [ ]:
statistics_df.to_excel(os.path.join(ground_plot_folder, 'trends', "statistics.xlsx"))

#### ACF and PACF

In [ ]:
for station_id in ground_df['Station'].unique():
    station_df = ground_df[ground_df['Station'] == station_id].copy()
    
    for column in station_df.columns.difference(['DateTime', 'Station']):
        
        print(f"{station_id} - {column}")
        plot_acf(station_df[column], lags=20)
        
        
        plot_pacf(station_df[column], lags=20)
        
        plt.show()

#### Causality and Cointegration

"If two or more time-series are cointegrated, then there must be Granger causality between them - either one-way or in both directions. However, the converse is not true."

"So, if your data are cointegrated but you don't find any evidence of causality, you have a conflict in your results. (This might occur if your sample size is too small to satisfy the asymptotics that the cointegration and causality tests rely on.) If you have cointegration and find one-way causality, everything is fine. (You may still be wrong about there being no causality in the other direction.) If your data are not cointegrated, then you have no cross-check on your causality results."

In [ ]:
# Function to perform Johansen test
def johansen_test(data, det_order, k_ar_diff=1):
    """
    Performs the Johansen cointegration test and prints the results.
    
    Parameters:
    data (numpy.ndarray): The time series data for the cointegration test.
    det_order (int): The order of the deterministic terms.
                     -1: No constant or trend.
                      0: Constant term only.
                      1: Constant and trend terms.
    k_ar_diff (int): The number of lags to include in the VAR model.
    """
    result = coint_johansen(data, det_order, k_ar_diff)
    
    print(f'Johansen Test Results (det_order={det_order})')
    print('Trace Statistics:', result.trace_stat)
    print('Critical Values (Trace):', result.trace_stat_crit_vals)
    print()

In [ ]:
def check_stationarity(df):
    """
    Check and make the time series stationary if required.
    """
    for column in df.columns:
        adf = adfuller(df[column])
        kp = kpss(df[column])
        if adf[1] > 0.05 and kp[1] < 0.05:
            print(f'{column} is non-stationary. Differencing the data.')
            df[column] = df[column].diff().dropna()
    return df.dropna()

In [ ]:
coint_dfs = {}

for station_id in ground_df['Station'].unique():
    station_df = ground_df[ground_df['Station'] == station_id].copy()
    
    station_df = station_df.drop(columns=diff_columns).dropna()
    
    print(f"Station {station_id}")
    print()
    
    df = check_stationarity(station_df)
    
    coint_df = pd.DataFrame(columns=df.columns, index=df.columns)
    
    # perfrom Engle-Granger test
    for column in df.columns:
        for column2 in df.columns:
            if column == column2:
                continue
            result = coint(df[column], df[column2])
            
            if result[1] < 0.05:
                coint_df.loc[column, column2] = True
            else:
                coint_df.loc[column, column2] = False
                
    coint_dfs[station_id] = coint_df

In [ ]:
coint_df

# the row is cointrated with the column

In [ ]:
# %%script false --no-raise-error
for station_id in ground_df['Station'].unique():
    station_df = ground_df[ground_df['Station'] == station_id].copy()
    
    station_df = station_df.drop(columns=diff_columns).dropna()
    
    station_df = check_stationarity(station_df)
    
    aic, bic, fpe, hqic = [], [], [], []
    model = VAR(station_df)
    if station_id == 105:
        p = range(0, 7)
    else:
        p = range(0, 17)
    for i in p:
        result = model.fit(i)
        aic.append(result.aic)
        bic.append(result.bic)
        fpe.append(result.fpe)
        hqic.append(result.hqic)
    lags_metrics_df = pd.DataFrame({'AIC': aic, 
                                    'BIC': bic, 
                                    'HQIC': hqic,
                                    'FPE': fpe}, 
                                index=p)    
    fig, ax = plt.subplots(1, 4, figsize=(15, 3), sharex=True)
    lags_metrics_df.plot(subplots=True, ax=ax, marker='o')
    
    # set the title
    plt.suptitle(f"Station {station_id}")
    
    plt.tight_layout()
    

In [ ]:
# select max lag = 1 for all stations

In [ ]:
# multivariate case

def check_stationarity(df):
    """
    Check and make the time series stationary if required.
    """
    for column in df.columns:
        adf = adfuller(df[column])
        kp = kpss(df[column])
        if adf[1] > 0.05 and kp[1] < 0.05:
            print(f'{column} is non-stationary. Differencing the data.')
            df[column] = df[column].diff().dropna()
    return df.dropna()

def grangers_causation_matrix_multivariate(data, maxlag=12, test='ssr_chi2test', verbose=False):
    """
    Check Granger Causality in a multivariate setting.
    """
    variables = data.columns
    df = pd.DataFrame(np.zeros((len(variables), len(variables))), columns=variables, index=variables)
        
    # Ensure data is stationary
    data = check_stationarity(data)
    
    # Select the optimal lag length for the VAR model
    
    # Fit the VAR model
    model = VAR(data)
    model_fitted = model.fit(maxlags=maxlag)
    
    for r in variables:
        for c in variables:
            if c != r:
                test_result = model_fitted.test_causality(r, c, kind='f')
                p_value = round(test_result.pvalue, 4)
                df.loc[r, c] = p_value
                if verbose:
                    print(f'Y = {r}, X = {c}, P-Value = {p_value}')
            else:
                df.loc[r, c] = 1
    
    df.columns = [var + '_x' for var in variables]
    df.index = [var + '_y' for var in variables]
    
    return df, model_fitted.summary(), model_fitted.irf(12)


In [ ]:
causality_matrices = {}

for station_id in ground_df['Station'].unique():
    station_df = ground_df[ground_df['Station'] == station_id].copy()
    
    causality_matrix, summary, irf = grangers_causation_matrix_multivariate(station_df.drop(columns=diff_columns).dropna(), maxlag=1)
    
    causality_matrices[station_id] = causality_matrix
    
    print(f"Station {station_id}")
    print(summary)
    
    ax = irf.plot(orth=True)
    ax.set_size_inches(40, 20)
    
    plt.show()
    

In [ ]:
# check if the cointration matrix is consistent with the Granger Causality matrix
# if the cointration matrix is True, then the Granger Causality matrix should have a p-value < 0.05

for station_id in ground_df['Station'].unique():
    
    coint_df = coint_dfs[station_id]
    causality_matrix = causality_matrices[station_id]
    
    print(f"Station {station_id}")
    print()
    
    count = 0
    cons_count = 0
    
    for column in coint_df.columns:
        for column2 in coint_df.columns:
            
            if column == column2:
                continue
            
            count += 1
            
            if coint_df.loc[column, column2]:
                if causality_matrix.loc[column + '_y', column2 + '_x'] < 0.05:
                    print(f"{column} -> {column2} is consistent")
                    cons_count += 1
                else:
                    print(f"{column} -> {column2} is not consistent")
            else:
                if causality_matrix.loc[column + '_y', column2 + '_x'] < 0.05:
                    print(f"{column} -> {column2} is not consistent")
                else:
                    print(f"{column} -> {column2} is consistent")
                    cons_count += 1
                    
    print()
    print()
    
    print(f"Consistency: {cons_count}/{count}")

In [ ]:
for station_id, causality_matrix in causality_matrices.items():
    
    fig = go.Figure()
    
    fig.add_trace(
        go.Heatmap(
            z=causality_matrix.values,
            x=causality_matrix.columns,
            y=causality_matrix.index,
        )
    )
    
    annotations = []
    for i in range(causality_matrix.shape[0]):
        for j in range(causality_matrix.shape[1]):
            
            if i == j:
                continue
            
            value = causality_matrix.iloc[i, j]
            if value < 0.05:
                annotations.append(
                    dict(
                        x=causality_matrix.columns[j],
                        y=causality_matrix.index[i],
                        text=f"{value:.2f}",
                        showarrow=False,
                        font=dict(color="white"),
                        xref="x",
                        yref="y"
                    )
                )
    
    fig.update_layout(
        title=f"Granger Causality p-value Matrix - Station {station_id}",
        xaxis_title="X",
        yaxis=dict(
            title="Y",
            autorange='reversed'  # Reverse the order of the y-axis
        ),
        annotations= annotations + [
                dict(
                    xref='paper',
                    yref='paper',
                    x=-0.2,
                    y=-0.6,
                    showarrow=False,
                    text='Note: if a given p-value is < significance level (0.05), then, <br> the corresponding X series (column) causes the Y (row).',
                    font=dict(
                        size=12
                    )
                )
            ]
    )
    
    fig.show()

In [ ]:
# store the results in an excel file
for station_id, causality_matrix in causality_matrices.items():
    causality_matrix.to_excel(os.path.join(ground_plot_folder, 'granger', f"{station_id}.xlsx"))

In [ ]:
# univariate case

def select_best_lag(data, maxlag):
    """
    Select the best lag length using information criteria.
    """
    model = VAR(data)
    lag_order = model.select_order()
    return lag_order.aic


# Check for stationarity using both adfuller and kpss tests
def check_stationarity(series):
    adf_result = adfuller(series)
    kpss_result = kpss(series)
    return adf_result[1] < 0.05 and kpss_result[1] < 0.05

def grangers_causation_matrix(data, variables, maxlag=12, test='ssr_chi2test', verbose=False):
    df = pd.DataFrame(np.zeros((len(variables), len(variables))), columns=variables, index=variables)
    
    df = df.astype(object)
    
    lag_df = pd.DataFrame(np.zeros((len(variables), len(variables))), columns=variables, index=variables)
    
    for c in df.columns:
        for r in df.index:
            if c != r:
                granger_df = data[[c, r]]
                if data[c].isna().any() or data[r].isna().any():
                    granger_df = granger_df[[r, c]].dropna()
                
                try:
                    adfuller(granger_df[c])
                    adfuller(granger_df[r])
                except Exception as e:
                    print(e)
                    continue
                
                if not check_stationarity(granger_df[c]):
                    print(f"{c} is non-stationary")
                    print("Differencing data")
                    granger_df[c] = granger_df[c].diff().dropna()
                
                if not check_stationarity(granger_df[r]):
                    print(f"{r} is non-stationary")
                    print("Differencing data")
                    granger_df[r] = granger_df[r].diff().dropna()
                    
                granger_df.dropna(inplace=True)
                
                # Select the best lag
                selected_lag = select_best_lag(granger_df, maxlag)
                
                test_result = grangercausalitytests(granger_df, maxlag=selected_lag, verbose=False)
                
                # p_values = [round(test_result[i+1][0][test][1], 4) for i in range(selected_lag)]
                
                # the lag is the index of the list + 1
                # store all the p-values to make a plot later
                p_values = [round(test_result[i+1][0][test][1], 4) for i in range(selected_lag)]
                statistics = [test_result[i+1][0][test][0] for i in range(selected_lag)]
                
                # need to store as list to avoid the error
                
                # store both the p values and the statistics in the same cell of df
                df.loc[r, c] = [p_values, statistics]
                
            else:
                df.loc[r, c] = 1
                
    df.columns = [var + '_x' for var in variables]
    df.index = [var + '_y' for var in variables]
    
    return df, lag_df


In [ ]:
causality_matrices = {}
lag_matrices = {}

for station_id in ground_df['Station'].unique():
    station_df = ground_df[ground_df['Station'] == station_id].copy()
    
    causality_matrix, lag_matrix = grangers_causation_matrix(station_df, variables=station_df.columns.difference(diff_columns), maxlag=12)
    
    causality_matrices[station_id] = causality_matrix
    lag_matrices[station_id] = lag_matrix

In [ ]:
causality_matrices[station_id]

In [ ]:
for station_id, causality_matrix in causality_matrices.items():
    
    for i in range(causality_matrix.shape[0]):
        for j in range(causality_matrix.shape[1]):
            if i != j:
                test_result = causality_matrix.iloc[i, j]
                
                # check if the test result is a list
                if not isinstance(test_result, list):
                    continue
                
                p_values = test_result[0]
                statistics = test_result[1]
                
                lags = np.arange(1, len(p_values) + 1)
                
                fig = go.Figure()
                
                fig.add_trace(
                    go.Scatter(
                        x=lags,
                        y=p_values,
                        mode='lines',
                        name='P-Value'
                    )
                )
                
                # fig.add_trace(
                #     go.Scatter(
                #         x=lags,
                #         y=statistics,
                #         mode='lines',
                #         name='F-Statistic'
                #     )
                # )
                
                fig.update_layout(
                    title=f"Station {station_id} - Granger Causality Test - {causality_matrix.columns[j]} -> {causality_matrix.index[i]}",
                    xaxis_title="Lag",
                    yaxis_title="Value"
                )
                
                fig.show()              

In [ ]:
for station_id, lag_matrix in lag_matrices.items():
    
    px.imshow(
        lag_matrix,
        labels=dict(x="X", y="Y", color="Lag"),
        title=f"Lag Matrix - Station {station_id}",
        text_auto=True,
        color_continuous_scale='Viridis',
        width=800,
        height=800
    ).show()

### Data Analysis

#### TS Decomposition

In [ ]:
color = 'rgb(200, 2, 110)'

In [ ]:
for station_id in ground_df['Station'].unique():
    station_df = ground_df[ground_df['Station'] == station_id].copy()
    
    station_df.set_index('DateTime', inplace=True)
    
    for column in station_df.columns.difference(diff_columns):
        
        df = pd.DataFrame({
            'ds': station_df.index,
            'y': station_df[column]
        })
        
        model = Prophet(weekly_seasonality=False, daily_seasonality=False)
        model.fit(df)
        # Make predictions for both columns
        future = model.make_future_dataframe(periods=0)
        forecast = model.predict(future)
        
        fig = go.Figure()

        fig.add_trace(
            go.Scatter(
                x=station_df.index,
                y=station_df[column],
                mode='lines',
                name='Historical',
                line=dict(
                    color='black',
                    width=0.6
                )
            )
        )
        
        fig.add_trace(
            go.Scatter(
                x=forecast['ds'],
                y=forecast['trend'],
                mode='lines',
                name='Trend',
                line=dict(color=color)
            )
        )
        
        fig.update_layout(
            xaxis_title="Time",
            yaxis_title=f"{column}"
        )
        
        if not os.path.exists(os.path.join(ground_plot_folder, 'trends', f"station_{station_id}")):
            os.makedirs(os.path.join(ground_plot_folder, 'trends', f"station_{station_id}")) 
            
        column_ = column.replace("/", "_")
        
        fig.write_image(
            os.path.join(
                ground_plot_folder,
                'trends',
                f"station_{station_id}",
                f"{column_}.png"
            ),
            scale=3
        )

##### Monthly

In [ ]:
# Perform trend analysis for each month

# Build interaction plot

months = ground_df['DateTime'].dt.month.unique()
months.sort()

# Get the names of the months
months_name = [pd.to_datetime(f"{month}-01-2021").strftime("%B") for month in months]

# Create dataframe to store the adf and mann-kendall test results for each station for each month
statistics_df = pd.DataFrame(
    index=pd.MultiIndex.from_product([ground_df.columns.difference(diff_columns), months_name]),
    columns=pd.MultiIndex.from_product([ground_df['Station'].unique().tolist(), ['ADF p-value', 'ADF result', 'MK p-value', 'MK result', 'Slope', 'Slope p-value', 'Trend Percentage']])
)

legend_added = set()

# Calculate global min and max for each variable
global_min_max = {}
for column in ground_df.columns.difference(diff_columns):
    global_min_max[column] = (ground_df[column].min(), ground_df[column].max())

for station_id in ground_df['Station'].unique():
    
    station_df = ground_df[ground_df['Station'] == station_id].copy()
    
    station_df.set_index('DateTime', inplace=True)
    
    # Analysis post Berlin Wall Fall
    station_df = station_df[station_df.index >= "1992-01-01"]
    
    for column in station_df.columns.difference(diff_columns):
        
        fig = make_subplots(
            4,
            3,
            subplot_titles=months_name,
            vertical_spacing=0.08,  # Reduce vertical spacing
            horizontal_spacing=0.05  # Reduce horizontal spacing
        )
        
        for i, month in enumerate(months):
            
            month_df = station_df[station_df.index.month == month]
                       
            # Perform Augmented Dickey-Fuller test
            try:
                adf_result = adfuller(month_df[column].dropna(), autolag="AIC")
            except Exception as e:
                print(e)
                continue
            
            # Perform Mann-Kendall test
            mk_result = mk.original_test(month_df[column])
            
            # Store the results in the statistics dataframe
            statistics_df.loc[(column, months_name[i]), (station_id, 'ADF p-value')] = adf_result[1]
            statistics_df.loc[(column, months_name[i]), (station_id, 'ADF result')] = 'Stationary' if adf_result[1] < 0.05 else 'Non-Stationary'
            
            statistics_df.loc[(column, months_name[i]), (station_id, 'MK p-value')] = mk_result.p
            statistics_df.loc[(column, months_name[i]), (station_id, 'MK result')] = mk_result.trend
            
            df = pd.DataFrame({
                'ds': month_df.index,
                'y': month_df[column]
            })
            
            model = Prophet(yearly_seasonality=False, weekly_seasonality=False, daily_seasonality=False)
            model.fit(df)
            # Make predictions for both columns
            future = model.make_future_dataframe(periods=0)
            forecast = model.predict(future)
            
            # compute linear regression on trend
            X = np.arange(df.shape[0])
            X = sm.add_constant(X)
            y = df["y"].copy()

            model = sm.OLS(y, X)
            results = model.fit()
            
            slope = results.params.iloc[1]

            p_value = results.pvalues.iloc[1]
            
            # store the slope
            statistics_df.loc[(column, months_name[i]), (station_id, 'Slope')] = slope
            statistics_df.loc[(column, months_name[i]), (station_id, 'Slope p-value')] = p_value
            
            # calculate the percentage of the trend
            first_value = forecast['trend'].iloc[0]
            last_value = forecast['trend'].iloc[-1]
            
            trend_percentage = (last_value - first_value) / first_value * 100
            
            statistics_df.loc[(column, months_name[i]), (station_id, 'Trend Percentage')] = trend_percentage
                
            fig.add_trace(
                go.Scatter(
                    x=month_df.index,
                    y=month_df[column],
                    mode='lines',
                    name='Historical',
                    line=dict(
                        color='black',
                        width=0.6
                    ),
                    showlegend=True if column not in legend_added else False
                ),
                row=(i // 3) + 1,
                col=(i % 3) + 1
            )
            
            fig.add_trace(
                go.Scatter(
                    x=forecast['ds'],
                    y=forecast['trend'],
                    mode='lines',
                    name='Trend',
                    line=dict(color='blue'),
                    showlegend=True if column not in legend_added else False
                ),
                row=(i // 3) + 1,
                col=(i % 3) + 1
            )
            
            legend_added.add(column)
            
        # Set the same y-axis range for all subplots
        fig.update_yaxes(range=global_min_max[column])
        
        fig.add_annotation(dict(
            x=0.5,
            y=-0.08,
            showarrow=False,
            text="Time",
            xref="paper",
            yref="paper",
            font=dict(size=20)
        ))

        fig.add_annotation(dict(
            x=-0.08,
            y=0.5,
            showarrow=False,
            text=column,
            textangle=-90,
            xref="paper",
            yref="paper",
            font=dict(size=20)
        )) 
            
        fig.update_layout(
            height=1000,
            width=1200,
            title={
                'text': f"Station {station_id}",
                'y': 0.98,  # Vertical position
                'x': 0.5,  # Horizontal position
                'xanchor': 'center',
                'yanchor': 'top'
            },
            font=dict(size=18),
            margin=dict(t=80, l=80, b=100, r=40)  # Adjust margins
        )

        if not os.path.exists(os.path.join(ground_plot_folder, 'trends', 'monthwise', f"station_{station_id}")):
            os.makedirs(os.path.join(ground_plot_folder, 'trends', 'monthwise', f"station_{station_id}")) 
            
        column_ = column.replace("/", "_")
        
        fig.write_image(
            os.path.join(
                ground_plot_folder,
                'trends',
                'monthwise',
                f"station_{station_id}",
                f"{column_}.png"
            ),
            scale=3
        )

In [ ]:
statistics_df.to_excel(os.path.join(ground_plot_folder, 'trends', 'monthwise', "statistics.xlsx"))

#### Correlation

##### Cross-Correlation (lags)

In [ ]:
def check_stationarity(df):
    """
    Check and make the time series stationary if required.
    """
    for column in df.columns:
        adf = adfuller(df[column])
        kp = kpss(df[column])
        if adf[1] > 0.05 and kp[1] < 0.05:
            print(f'{column} is non-stationary. Differencing the data.')
            df[column] = df[column].diff().dropna()
    return df.dropna()

In [ ]:
def compute_cross_corr(ts1, ts2, max_lag):
    lags = range(-max_lag, max_lag + 1)
    cross_corr = [ts1.corr(ts2.shift(lag)) for lag in lags]
    return lags, cross_corr

In [ ]:
# check if a time series is stationary, if not, difference it

correlation_results_df = pd.DataFrame(
    index=ground_df.columns.difference(diff_columns),
    columns=pd.MultiIndex.from_product([ground_df['Station'].unique().tolist(), ground_df.columns.difference(diff_columns)])
)

for station_id in ground_df['Station'].unique():
    station_df = ground_df[ground_df['Station'] == station_id].copy()
    
    station_df = station_df.drop(columns=diff_columns).dropna()
    
    station_df = check_stationarity(station_df)
    
    # normalize the data
    for column in station_df.columns:
        scaler = MinMaxScaler()
        station_df[column] = scaler.fit_transform(station_df[[column]])
        
    
    max_lag = 12
    
    for column in station_df.columns:
        for column2 in station_df.columns:
            if column == column2:
                continue
            
            lags, cross_corr = compute_cross_corr(station_df[column], station_df[column2], max_lag)
            
            fig = go.Figure()
            
            fig.add_trace(
                go.Scatter(
                    x=list(lags),
                    y=cross_corr,
                    mode='lines+markers',
                    name='Cross Correlation'
                )
            )
            
            fig.update_layout(
                title=f"Station {station_id} - {column} vs {column2}",
                xaxis_title="Lag (Months)",
                yaxis_title="Cross Correlation"
            )
            
            if not os.path.exists(os.path.join(ground_cross_corr_folder, f"station_{station_id}")):
                os.makedirs(os.path.join(ground_cross_corr_folder, f"station_{station_id}"))
                
                
            column_ = column.replace("/", "_")
            column2_ = column2.replace("/", "_")
                
            fig.write_image(
                os.path.join(
                    ground_cross_corr_folder,
                    f"station_{station_id}",
                    f"{column_}_vs_{column2_}.png"
                ),
                scale=3
            )
            
            corr, pvalue = pearsonr(station_df[column], station_df[column2])
            
            # put the correlation just if the p-value is less than 0.05 and in the upper triangle
            if pvalue < 0.05:
                correlation_results_df.loc[column, (station_id, column2)] = corr
            else:
                correlation_results_df.loc[column, (station_id, column2)] = np.nan
                
            

In [ ]:
correlation_results_df.to_excel(os.path.join(ground_cross_corr_folder, "correlation_results.xlsx"))

##### Year by Year Correlation

In [ ]:
# perform year by year correlation
correlation_results = pd.DataFrame(
    index=pd.MultiIndex.from_product([ground_df.columns.difference(diff_columns), sorted(ground_df['DateTime'].dt.year.unique())]),
    columns=pd.MultiIndex.from_product([ground_df['Station'].unique().tolist(), ground_df.columns.difference(diff_columns)])
)

In [ ]:
for station_id in ground_df['Station'].unique():
    station_df = ground_df[ground_df['Station'] == station_id].copy()
    
    # normalize the data
    for column in station_df.columns.difference(diff_columns):
        scaler = MinMaxScaler()
        station_df[column] = scaler.fit_transform(station_df[[column]])
    
    correlation_results[station_id] = {}
    
    for year in station_df['DateTime'].dt.year.unique():
        
        year_df = station_df[station_df['DateTime'].dt.year == year]
        
        year_df.drop(columns=diff_columns, inplace=True)
        
        for column in year_df.columns:
            for column2 in year_df.columns:
                if column == column2:
                    continue
                
                result = pearsonr(year_df[column], year_df[column2])
                
                correlation_results.loc[(column, year), (station_id, column2)] = result

In [ ]:
# plot the correlation results

for station_id in ground_df['Station'].unique():
        
        station_df = ground_df[ground_df['Station'] == station_id].copy()
        
        station_df.set_index('DateTime', inplace=True)
        
        for column in station_df.columns.difference(diff_columns):
            
            fig = go.Figure()
            
            for column2 in station_df.columns.difference(diff_columns):
                
                if column == column2:
                    continue
                
                years = correlation_results.loc[column, (station_id, column2)].index
                
                correlation = correlation_results.loc[(column, years), (station_id, column2)].dropna()            
                
                fig.add_trace(
                    go.Scatter(
                        x=correlation.index.get_level_values(1),
                        y=correlation.apply(lambda x: x[0]),
                        mode='lines+markers',
                        name=column2
                    )
                )
                
            fig.update_layout(
                title=f"Station {station_id} - {column} vs Other Parameters",
                xaxis_title="Year",
                yaxis_title="Pearson Correlation Coefficient"
            )
            
            
            if not os.path.exists(os.path.join(ground_corr_folder, f"station_{station_id}")):
                os.makedirs(os.path.join(ground_corr_folder, f"station_{station_id}"))
                
            column_ = column.replace("/", "_")
            
            fig.write_image(
                os.path.join(
                    ground_corr_folder,
                    f"station_{station_id}",
                    f"{column_}.png"
                ),
                scale=3,
                width=1200,
            )
            
            fig.show()

In [ ]:
correlation_results.to_excel(os.path.join(ground_corr_folder, "correlation_results.xlsx"))

#### Regression for Average Variations

##### Month by Month Variation for each Year

A time series of each year is built, the trend is computed with Prophet and a linear regression on the trend is computed in order to get the slope, which is the average month variation for the given year. 

In [ ]:
# perform trend analysis for each year

# build interaction plot

years = ground_df['DateTime'].dt.year.unique()
years.sort()

# convert the years to int
years = [int(year) for year in years]

# create dataframe to store the adf and mann-kendall test results for each station for each year
slopes_df = pd.DataFrame(
    index=pd.MultiIndex.from_product([ground_df.columns.difference(diff_columns), years]),
    columns=pd.MultiIndex.from_product([ground_df['Station'].unique().tolist(), ['Slope', 'P-Value', 'Error', 'Trend Percentage']])
)

legend_added = set()

for station_id in ground_df['Station'].unique():
    
    station_df = ground_df[ground_df['Station'] == station_id].copy()
    
    station_df.set_index('DateTime', inplace=True)
    
    station_df.dropna(inplace=True)
    
    for column in station_df.columns.difference(diff_columns):
        
        # fig = make_subplots(
        #     4,
        #     3,
        #     subplot_titles=years,
        #     vertical_spacing=0.08,  # Reduce vertical spacing
        #     horizontal_spacing=0.05  # Reduce horizontal spacing
        # )
        
        for i, year in enumerate(years):
            
            if year not in station_df.index.year:
                continue
            
            year_df = station_df[station_df.index.year == year]
            
            df = pd.DataFrame({
                'ds': year_df.index,
                'y': year_df[column]
            })
            
            model = Prophet(yearly_seasonality=False, weekly_seasonality=False, daily_seasonality=False)
            model.fit(df)
            # Make predictions for both columns
            future = model.make_future_dataframe(periods=0)
            forecast = model.predict(future)
            
            # compute the linear regression on the trend
            X = np.arange(forecast.shape[0])
            X = sm.add_constant(X)
            y = forecast['trend'].copy()
            
            model = sm.OLS(y, X)
            results = model.fit()
            
            slope = results.params.iloc[1]
            p_value = results.pvalues.iloc[1]
            st_error = results.bse.iloc[1]
            
            # store the results in the dataframe
            slopes_df.loc[(column, year), (station_id, 'Slope')] = round(slope, 4)
            slopes_df.loc[(column, year), (station_id, 'P-Value')] = round(p_value, 4)
            slopes_df.loc[(column, year), (station_id, 'Error')] = round(st_error, 4)
            
            # compute the percentage of the trend
            trend_percentage = (slope / forecast['trend'].mean()) * 100
            slopes_df.loc[(column, year), (station_id, 'Trend Percentage')] = round(trend_percentage, 4)

In [ ]:
slopes_df.to_excel(os.path.join(ground_plot_folder, 'trends', "slopes.xlsx"))

##### Year by Year Variation for each Month

A time series of each month is built, the trend is computed with Prophet and a linear regression on the trend is computed in order to get the slope, which is the average year variation for the given month. 

In [ ]:
# perform trend analysis for each month

# build interaction plot

months = ground_df['DateTime'].dt.month.unique()
months.sort()

# get the names of the months
months_name = [pd.to_datetime(f"{month}-01-2021").strftime("%B") for month in months]

# create dataframe to store the adf and mann-kendall test results for each station for each month
slopes_df = pd.DataFrame(
    index=pd.MultiIndex.from_product([ground_df.columns.difference(diff_columns), months_name]),
    columns=pd.MultiIndex.from_product([ground_df['Station'].unique().tolist(), ['Slope', 'P-Value', 'Error', 'Trend Percentage']])
)

legend_added = set()

for station_id in ground_df['Station'].unique():
    
    station_df = ground_df[ground_df['Station'] == station_id].copy()
    
    station_df.set_index('DateTime', inplace=True)
    
    for column in station_df.columns.difference(diff_columns):
        
        # fig = make_subplots(
        #     4,
        #     3,
        #     subplot_titles=months_name,
        #     vertical_spacing=0.08,  # Reduce vertical spacing
        #     horizontal_spacing=0.05  # Reduce horizontal spacing
        # )
        
        for i, month in enumerate(months):
            
            month_df = station_df[station_df.index.month == month]
            
            df = pd.DataFrame({
                'ds': month_df.index,
                'y': month_df[column]
            })
            
            model = Prophet(yearly_seasonality=False, weekly_seasonality=False, daily_seasonality=False)
            model.fit(df)
            # Make predictions for both columns
            future = model.make_future_dataframe(periods=0)
            forecast = model.predict(future)
            
            # compute the linear regression on the trend
            X = np.arange(forecast.shape[0])
            X = sm.add_constant(X)
            y = forecast['trend'].copy()
            
            model = sm.OLS(y, X)
            results = model.fit()
            
            slope = results.params.iloc[1]
            p_value = results.pvalues.iloc[1]
            st_error = results.bse.iloc[1]
            
            # store the results in the dataframe
            slopes_df.loc[(column, months_name[i]), (station_id, 'Slope')] = slope
            slopes_df.loc[(column, months_name[i]), (station_id, 'P-Value')] = p_value
            slopes_df.loc[(column, months_name[i]), (station_id, 'Error')] = st_error
            
            # compute the percentage of the trend
            trend_percentage = (slope / forecast['trend'].mean()) * 100
            slopes_df.loc[(column, months_name[i]), (station_id, 'Trend Percentage')] = trend_percentage
            

In [ ]:
slopes_df.to_excel(os.path.join(ground_plot_folder, 'trends', 'monthwise', "slopes.xlsx"))

In [ ]:
%%script false --no-raise-error
# per il momento lascia qua, potrebbe tornare utile se si decidesse di calcolare una linear regression tra ogni changepoint

for station_id in ground_df['Station'].unique():
    
    station_df = ground_df[ground_df['Station'] == station_id].copy()
    
    station_df.set_index('DateTime', inplace=True)
    
    for column in station_df.columns.difference(diff_columns):
        
        df = pd.DataFrame(
            {
                'ds': station_df.index,
                'y': station_df[column]
            }
        )
        
        model = Prophet(weekly_seasonality=False, daily_seasonality=False)
        model.fit(df)
        
        future = model.make_future_dataframe(periods=0)
        forecast = model.predict(future)
        
        # add the changepoints detected by Prophet
        changepoints = model.changepoints
    
        signif_changepoints = changepoints[
            np.abs(np.nanmean(model.params['delta'], axis=0)) >= 0.1
        ]
        
        changepoints_values = forecast.loc[forecast['ds'].isin(signif_changepoints), 'trend']
        
        trend = forecast['trend']
        
        # compute linear regression on trend
        X = np.arange(trend.shape[0])
        X = sm.add_constant(X)
        y = trend.copy()
        
        model = sm.OLS(y, X)
        results = model.fit()
        
        # plot the line of the linear regression
        line = pd.Series(results.predict(X), index=forecast['ds'])
        
        # get the slope with the p-value and the standard error
        slope = results.params.iloc[1]
        p_value = results.pvalues.iloc[1]
        std_err = results.bse.iloc[1]
        
        fig = go.Figure()
        
        fig.add_trace(
            go.Scatter(
                x=station_df.index,
                y=station_df[column],
                mode='lines',
                name='Original',
                line=dict(color='black')
            )
        )
        
        fig.add_trace(
            go.Scatter(
                x=forecast['ds'],
                y=trend,
                mode='lines',
                name='Trend',
                line=dict(color=color)
            )
        )
        
        fig.add_trace(
            go.Scatter(
                x=line.index,
                y=line,
                mode='lines',
                name='Linear Regression',
                line=dict(color='red', dash='dash')
            )
        )
        
        fig.add_trace(
            go.Scatter(
                x=signif_changepoints,
                y=changepoints_values,
                mode='markers',
                name='Changepoints',
                marker=dict(
                    color='green',
                    size=10
                )
            )
        )
        
        fig.update_layout(
            title=f"Slope: {slope:.3f}, P-Value: {p_value:.3f}, Std. Error: {std_err:.3f}",
            xaxis_title="Date",
            yaxis_title="Value"
        )

        fig.show()
        

# Class Frequency Analysis

## Rainfall

In [ ]:
rainfall_df = surface_df[surface_df['Station'] == 105][['DateTime', 'Cumulated Rainfall (mm)']].copy()

# define classes as [0,1], (1, 2], (2, 3], (3, inf)

rainfall_df['Class'] = pd.cut(rainfall_df['Cumulated Rainfall (mm)'], bins=[0, 1, 2, 3, np.inf], labels=['0-1', '1-2', '2-3', '3+'])

rainfall_df['Year'] = rainfall_df['DateTime'].dt.year
rainfall_df['Month'] = rainfall_df['DateTime'].dt.month

# analyze if the frequency of the classes changes over time

# create a pivot table
pivot_table = rainfall_df.pivot_table(index='Year', columns='Class', aggfunc='size', fill_value=0)

# plot the pivot table
fig = go.Figure()

for column in pivot_table.columns:
    fig.add_trace(
        go.Scatter(
            x=pivot_table.index,
            y=pivot_table[column],
            mode='lines+markers',
            name=column
        )
    )
    
fig.update_layout(
    title="Rainfall Classes Frequency Over Time",
    xaxis_title="Year",
    yaxis_title="Frequency"
)

fig.show()

## Air Temperature

In [ ]:
# plot the air temperature

air_temperature_df = surface_df[surface_df['Station'] == 305][['DateTime', 'Air Temperature (°C)']].copy()

air_temperature_df['Class'] = pd.cut(air_temperature_df['Air Temperature (°C)'], bins=[-10, 0, 10, 20, 30, 40, np.inf], labels=['-10-0', '0-10', '10-20', '20-30', '30-40', '40+'])

air_temperature_df['Year'] = air_temperature_df['DateTime'].dt.year
air_temperature_df['Month'] = air_temperature_df['DateTime'].dt.month

# analyze if the frequency of the classes changes over time

# create a pivot table
pivot_table = air_temperature_df.pivot_table(index='Year', columns='Class', aggfunc='size', fill_value=0)

# plot the pivot table
fig = go.Figure()

for column in pivot_table.columns:
    fig.add_trace(
        go.Scatter(
            x=pivot_table.index,
            y=pivot_table[column],
            mode='lines+markers',
            name=column
        )
    )
    
fig.update_layout(
    title="Air Temperature Classes Frequency Over Time",
    xaxis_title="Year",
    yaxis_title="Frequency"
)

fig.show()


In [ ]:
import math

def largest_factors(x):
    # Start with the largest integer close to sqrt(x)
    a = math.isqrt(x)
    
    # Look for the largest divisor of x starting from a
    while x % a != 0:
        a -= 1
    
    # Compute the other factor
    b = x // a
    return a, b

In [ ]:
air_temp_df = surface_df[surface_df['Station'] == 305][['DateTime', 'Air Temperature (°C)']].copy()

# define classes as [0,1], (1, 2], (2, 3], (3, inf)

# create a plot where for every year the histogram distribution of the air temperature is shown

n_years = air_temp_df['DateTime'].dt.year.nunique()

rows, cols = largest_factors(n_years)

fig = make_subplots(
    rows=rows,
    cols=cols,
    subplot_titles=[str(year) for year in air_temp_df['DateTime'].dt.year.unique()],  # Convert year to string
    vertical_spacing=0.08,  # Reduce vertical spacing
    horizontal_spacing=0.05  # Reduce horizontal spacing
)

for i, year in enumerate(air_temp_df['DateTime'].dt.year.unique()):
        
        year_df = air_temp_df[air_temp_df['DateTime'].dt.year == year]
        
        fig.add_trace(
            go.Histogram(
                x=year_df['Air Temperature (°C)'],
                name=str(year),
                opacity=0.75,
                nbinsx=10,
                marker=dict(
                    color='blue'
                )
            ),
            row=(i // cols) + 1,
            col=(i % cols) + 1
        )
        
fig.update_layout(
    height=1000,
    width=1200,
    title={
        'text': f"Station 305 - Air Temperature Distribution",
        'y': 0.98,  # Vertical position
        'x': 0.5,  # Horizontal position
        'xanchor': 'center',
        'yanchor': 'top'
    },
    font=dict(size=18),
    margin=dict(t=80, l=80, b=100, r=40), # Adjust margins
    showlegend=False
)

# Plots for the paper

In [ ]:
width_pixels = 90 * 3.779527559

## Scatter Plot

For the ground water, it is in the raw_data.ipynb

In [ ]:
# scatter plot between DOC nad Ammonium with best fit line

scatter_plots_folder = os.path.join(paper_plots_folder, 'Berlin', 'Scatters', 'Surface')

for station_id in surface_df['Station'].unique():
    doc_ammonium_df = surface_df[surface_df['Station'] == station_id][['DateTime', 'DOC (mg/l)', 'Ammonium (mg/l)']].copy()

    fig = px.scatter(
        doc_ammonium_df,
        x='Ammonium (mg/l)',
        y='DOC (mg/l)',
        trendline='ols',
        trendline_color_override='red'
    )

    fig.update_layout(
        title=f"Scatter Plot - DOC (mg/l) vs Ammonium (mg/l) - Station {station_id}",
        xaxis_title="Ammonium (mg/l)",
        yaxis_title="DOC (mg/l)"
    )

    fig.write_image(
        os.path.join(scatter_plots_folder, f"station_{station_id}.png"),
        scale=6,
    )
        

    # compute the correlation between DOC and Ammonium

    doc_ammonium_df.dropna(inplace=True)

    corr, pvalue = pearsonr(doc_ammonium_df['DOC (mg/l)'], doc_ammonium_df['Ammonium (mg/l)'])

    print(f"Pearson Correlation: {corr:.3f}, P-Value: {pvalue:.3f}")

## Trends

In [ ]:
width_pixels = 170 * 3.779527559

### Surface

In [ ]:
color = 'rgb(200, 2, 110)'

In [ ]:
# one plot for each variable
# do it just for the 305 station since it is the one used in the paper

trend_folder = os.path.join(paper_plots_folder, 'Berlin', 'Trends', 'Surface', '305')

# sort the columns into climatic, water quality and DOC as last one
climatic_variables = ['Air Temperature (°C)', 'Cumulated Rainfall (mm)']
water_quality_variables = ['Ammonium (mg/l)', 'Conductivity (µS/cm)', 'Dissolved Oxygen (mg/l)', 'Flow River Rate (m³/s)', 'Nitrate (mg/l)', 'pH', 'Water Temperature (°C)']
doc_variable = ['DOC (mg/l)']

columns = climatic_variables + water_quality_variables + doc_variable

colors = px.colors.qualitative.Plotly


color_mapping = {
    'Air Temperature (°C)': colors[0],
    'Cumulated Rainfall (mm)': colors[1],
    'Ammonium (mg/l)': colors[2],
    'Conductivity (µS/cm)': colors[3],
    'Dissolved Oxygen (mg/l)': colors[4],
    'Flow River Rate (m³/s)': colors[5],
    'Nitrate (mg/l)': colors[6],
    'pH': colors[7],
    'Water Temperature (°C)': colors[8],
    'DOC (mg/l)': colors[9]
}

surface_df = surface_df[surface_df['DateTime'] >= "1998-01-01"]


station_df = surface_df[surface_df['Station'] == station_id].copy()

station_df.set_index('DateTime', inplace=True)

station_df = station_df.drop(columns=bacteria_columns)

station_id = 305

for i, column in enumerate(columns):
    
    fig = go.Figure()
    
    df = pd.DataFrame({
        'ds': station_df.index,
        'y': station_df[column]
    })
    
    model = Prophet(weekly_seasonality=False, daily_seasonality=False)
    model.fit(df)
    # Make predictions for both columns
    future = model.make_future_dataframe(periods=0)
    forecast = model.predict(future)
    

    fig.add_trace(
        go.Scatter(
            x=station_df.index,
            y=station_df[column],
            mode='lines',
            name=column,
            line=dict(
                color=color_mapping[column],
                width=1.5
            ),
            showlegend=False
        ),
    )
    
    fig.add_trace(
        go.Scatter(
            x=forecast['ds'],
            y=forecast['trend'],
            mode='lines',
            name='Trend',
            line=dict(color=color, width=1),
            showlegend=False, # change to trend_show if you want to show the legend
            legendrank=np.inf
        ),
    )
    
        
    # fig.update_yaxes(title_text=column)
    
    if column == 'Ammonium (mg/l)':  # Replace with the actual column name
        fig.update_yaxes(range=[0, 0.7])
    
    if column == 'Air Temperature (°C)':
        fig.update_yaxes(range=[-10, 30])
    
    if column == 'Cumulated Rainfall (mm)':
        fig.update_yaxes(range=[0, 8])
        
    # if column == 'Conductivity (µS/cm)':
    #     fig.update_yaxes(range=[0, 1700])
    
    # if column == 'Nitrate (mg/l)':
    #     fig.update_yaxes(range=[0, 20])
    
    if column == 'pH':
        fig.update_yaxes(range=[7.6, 8.8])
    
    if column == 'Water Temperature (°C)':
        fig.update_yaxes(range=[0, 30])
        
    # if column == 'Flow River Rate (m³/s)':
    #     fig.update_yaxes(range=[0, 1300])
        
    if column == 'Dissolved Oxygen (mg/l)':
        fig.update_yaxes(range=[4, 18])


    start_year = station_df.index.year.min()
    end_year = station_df.index.year.max()
    tickvals = [pd.Timestamp(f'{year}-01-01') for year in range(start_year, end_year + 1, 4)]
    ticktext = [str(year) for year in range(start_year, end_year + 1, 4)]   

    fig.update_xaxes(
        tickvals=tickvals,
        ticktext=ticktext,
        title_text="Time"
    )
    
    fig.update_yaxes(title_text=column)  

    fig.update_layout(
        title=dict(
            text='Berlin',
            x=0.5,
            xanchor='center',
            yanchor='top',
        ),
        legend=dict(
            traceorder='normal',
        ),
    )

    #reduce the font size of the subplot titles
    for annotation in fig['layout']['annotations']:
        annotation['font'] = dict(size=8)

    
    column_ = column.replace("/", "_")

    fig.write_image(
        os.path.join(trend_folder, f"{column_}.png"),
        scale=10,
    )

    # fig.show()


In [ ]:
%%script false --no-raise-error
# one single image with all the trends

trend_folder = os.path.join(paper_plots_folder, 'Berlin', 'Trends', 'Surface')

# sort the columns into climatic, water quality and DOC as last one
climatic_variables = ['Air Temperature (°C)', 'Cumulated Rainfall (mm)']
water_quality_variables = ['Ammonium (mg/l)', 'Conductivity (µS/cm)', 'Dissolved Oxygen (mg/l)', 'Flow River Rate (m³/s)', 'Nitrate (mg/l)', 'pH', 'Water Temperature (°C)']
doc_variable = ['DOC (mg/l)']

columns = climatic_variables + water_quality_variables + doc_variable

colors = px.colors.qualitative.Plotly


color_mapping = {
    'Air Temperature (°C)': colors[0],
    'Cumulated Rainfall (mm)': colors[1],
    'Ammonium (mg/l)': colors[2],
    'Conductivity (µS/cm)': colors[3],
    'Dissolved Oxygen (mg/l)': colors[4],
    'Flow River Rate (m³/s)': colors[5],
    'Nitrate (mg/l)': colors[6],
    'pH': colors[7],
    'Water Temperature (°C)': colors[8],
    'DOC (mg/l)': colors[9]
}

surface_df = surface_df[surface_df['DateTime'] >= "1998-01-01"]

for station_id in surface_df['Station'].unique():
    station_df = surface_df[surface_df['Station'] == station_id].copy()
    
    station_df.set_index('DateTime', inplace=True)
    
    station_df = station_df.drop(columns=bacteria_columns)
    
    fig = make_subplots(
        len(columns),
        1,
        shared_xaxes=True,
        subplot_titles=columns,
        vertical_spacing=0.02
    )
    
    trend_show = True
    
    for i, column in enumerate(columns):
        
        df = pd.DataFrame({
            'ds': station_df.index,
            'y': station_df[column]
        })
        
        model = Prophet(weekly_seasonality=False, daily_seasonality=False)
        model.fit(df)
        # Make predictions for both columns
        future = model.make_future_dataframe(periods=0)
        forecast = model.predict(future)
        

        fig.add_trace(
            go.Scatter(
                x=station_df.index,
                y=station_df[column],
                mode='lines',
                name=column,
                line=dict(
                    color=color_mapping[column],
                    width=1.5
                ),
                legendrank=i,
                showlegend=False
            ),
            row=i + 1,
            col=1
        )
        
        fig.add_trace(
            go.Scatter(
                x=forecast['ds'],
                y=forecast['trend'],
                mode='lines',
                name='Trend',
                line=dict(color=color, width=1),
                showlegend=False, # change to trend_show if you want to show the legend
                legendrank=np.inf
            ),
            row=i + 1,
            col=1
        )
        
        if trend_show:
            trend_show = False
            
        # fig.update_yaxes(title_text=column, row=i + 1, col=1)
        
        if column == 'Ammonium (mg/l)':  # Replace with the actual column name
            fig.update_yaxes(range=[0, 0.7], row=i + 1, col=1)
        
        if column == 'Air Temperature (°C)':
            fig.update_yaxes(range=[-10, 30], row=i + 1, col=1)
        
        if column == 'Cumulated Rainfall (mm)':
            fig.update_yaxes(range=[0, 8], row=i + 1, col=1)
            
        # if column == 'Conductivity (µS/cm)':
        #     fig.update_yaxes(range=[0, 1700], row=i + 1, col=1)
        
        # if column == 'Nitrate (mg/l)':
        #     fig.update_yaxes(range=[0, 20], row=i + 1, col=1)
        
        if column == 'pH':
            fig.update_yaxes(range=[7.6, 8.8], row=i + 1, col=1)
        
        if column == 'Water Temperature (°C)':
            fig.update_yaxes(range=[0, 30], row=i + 1, col=1)
            
        # if column == 'Flow River Rate (m³/s)':
        #     fig.update_yaxes(range=[0, 1300], row=i + 1, col=1)
            
        if column == 'Dissolved Oxygen (mg/l)':
            fig.update_yaxes(range=[4, 18], row=i + 1, col=1)
        
        
    fig.update_xaxes(title_text="Time", row=len(columns), col=1)
    
    start_year = station_df.index.year.min()
    end_year = station_df.index.year.max()
    tickvals = [pd.Timestamp(f'{year}-01-01') for year in range(start_year, end_year + 1, 4)]
    ticktext = [str(year) for year in range(start_year, end_year + 1, 4)]   
    
    fig.update_xaxes(
        tickvals=tickvals,
        ticktext=ticktext,
    )  
    
    fig.update_layout(
        title=dict(
            text='Berlin',
            x=0.5,
            y=0.99,
            xanchor='center',
            yanchor='top',
            font=dict(size=10),
        ),
        font=dict(size=8),
        legend=dict(
            traceorder='normal',
        ),
        margin=dict(
        l=30,  # Left margin
        r=30,  # Right margin
        t=50,  # Top margin
        b=150  # Bottom margin (increase to add blank space)
        )
    )
    
    #reduce the font size of the subplot titles
    for annotation in fig['layout']['annotations']:
        annotation['font'] = dict(size=8)
    
    
    fig.write_image(
        os.path.join(trend_folder, f"station_{station_id}.png"),
        scale=10,
        width=400,
        height=220 * len(columns)
    )
    

In [ ]:
surface_df.columns.tolist()

### Ground

In [ ]:
ground_df.columns.tolist()

In [ ]:
trend_folder = os.path.join(paper_plots_folder, 'Berlin', 'Trends', 'Ground')

# sort the columns into climatic, water quality and DOC as last one
climatic_variables = ['Air Temperature (°C)', 'Cumulated Rainfall (mm)']
water_quality_variables = ['Ammonium (mg/l)', 'Conductivity (µS/cm)', 'Nitrate (mg/l)', 'pH', 'Water Temperature (°C)']
doc_variable = ['UVA254 (1/m)']

columns = climatic_variables + water_quality_variables + doc_variable

colors = px.colors.qualitative.Plotly

for station_id in ground_df['Station'].unique():
    station_df = ground_df[ground_df['Station'] == station_id].copy()
    
    station_df.set_index('DateTime', inplace=True)
    
    fig = make_subplots(
        len(columns),
        1,
        shared_xaxes=True,
        subplot_titles=columns,
    )
    
    trend_show = True
    
    for i, column in enumerate(columns):
        
        df = pd.DataFrame({
            'ds': station_df.index,
            'y': station_df[column]
        })
        
        model = Prophet(weekly_seasonality=False, daily_seasonality=False)
        model.fit(df)
        # Make predictions for both columns
        future = model.make_future_dataframe(periods=0)
        forecast = model.predict(future)
        

        fig.add_trace(
            go.Scatter(
                x=station_df.index,
                y=station_df[column],
                mode='lines',
                line=dict(
                    color=colors[i],
                    width=1.5
                ),
                legendrank=i,
                showlegend=False
            ),
            row=i + 1,
            col=1
        )
        
        fig.add_trace(
            go.Scatter(
                x=forecast['ds'],
                y=forecast['trend'],
                mode='lines',
                name='Trend',
                line=dict(color=color, width=1),
                showlegend=trend_show,
                legendrank=np.inf
            ),
            row=i + 1,
            col=1
        )
        
        if trend_show:
            trend_show = False
            
        # fig.update_yaxes(title_text=column, row=i + 1, col=1)
        
        # Set y-axis range for a specific trace (e.g., the first trace)
        if column == 'Ammonium (mg/l)':  # Replace with the actual column name
            fig.update_yaxes(range=[0, 0.7], row=i + 1, col=1)
        
        if column == 'Air Temperature (°C)':
            fig.update_yaxes(range=[-10, 30], row=i + 1, col=1)
        
        if column == 'Cumulated Rainfall (mm)':
            fig.update_yaxes(range=[0, 8], row=i + 1, col=1)
            
        if column == 'Conductivity (µS/cm)':
            fig.update_yaxes(range=[0, 1700], row=i + 1, col=1)
        
        if column == 'Nitrate (mg/l)':
            fig.update_yaxes(range=[0, 20], row=i + 1, col=1)
        
        if column == 'pH':
            fig.update_yaxes(range=[7.6, 8.8], row=i + 1, col=1)
        
        if column == 'Water Temperature (°C)':
            fig.update_yaxes(range=[0, 30], row=i + 1, col=1)
            
        if column == 'Flow River Rate (m³/s)':
            fig.update_yaxes(range=[0, 1300], row=i + 1, col=1)
        
        
            
        
    # # Add vertical dashed lines for each year
    # years = station_df.index.year.unique()
    # for year in years:
    #     fig.add_vline(
    #         x=pd.Timestamp(f'{year}-01-01'),
    #         line=dict(color='gray', width=0.5, dash='dash'),
    #         layer='above'
    #     )
        
    fig.update_xaxes(title_text="Time", row=len(columns), col=1)
    
    start_year = station_df.index.year.min()
    end_year = station_df.index.year.max()
    tickvals = [pd.Timestamp(f'{year}-01-01') for year in range(start_year, end_year + 1, 4)]
    ticktext = [str(year) for year in range(start_year, end_year + 1, 4)]   
    
    fig.update_xaxes(
        tickvals=tickvals,
        ticktext=ticktext,
    )  
    
    fig.update_layout(
        font=dict(size=8),
        legend=dict(
            traceorder='normal',
        ),
        
    )
    
    #reduce the font size of the subplot titles
    for annotation in fig['layout']['annotations']:
        annotation['font'] = dict(size=8)
    
    
    fig.write_image(
        os.path.join(trend_folder, f"station_{station_id}.png"),
        scale=10,
        width=500,
        height=150 * len(columns)
    )
    

## Correlation Year by Year

### Surface

In [ ]:
columns = ['Air Temperature (°C)', 'Ammonium (mg/l)', 'DOC (mg/l)']

# perform year by year correlation
correlation_results = pd.DataFrame(
    index=pd.MultiIndex.from_product([columns, sorted(surface_df['DateTime'].dt.year.unique())]),
    columns=pd.MultiIndex.from_product([surface_df['Station'].unique().tolist(), columns])
)

for station_id in surface_df['Station'].unique():
    station_df = surface_df[surface_df['Station'] == station_id].copy()
    
    station_df = station_df.drop(columns=['Flow River Rate (m³/s)'])
    
    station_df = station_df.drop(columns=bacteria_columns).dropna()
    
    # normalize the data
    for column in columns:
        scaler = MinMaxScaler()
        station_df[column] = scaler.fit_transform(station_df[[column]])
    
    correlation_results[station_id] = {}
    
    for year in station_df['DateTime'].dt.year.unique():
        
        year_df = station_df[station_df['DateTime'].dt.year == year]
        
        year_df = year_df[columns]
        
        for column in year_df.columns:
            for column2 in year_df.columns:
                if column == column2:
                    continue
                
                result = pearsonr(year_df[column], year_df[column2])
                
                correlation_results.loc[(column, year), (station_id, column2)] = result

In [ ]:
# plot the correlation results

correlation_folder = os.path.join(paper_plots_folder, 'Berlin', 'Correlations', 'Surface')

for station_id in surface_df['Station'].unique():
        
        station_df = surface_df[surface_df['Station'] == station_id].copy()
        
        station_df = station_df.drop(columns=bacteria_columns).dropna()
        
        station_df.set_index('DateTime', inplace=True)
        
        for column in columns:
            
            fig = go.Figure()
            
            for column2 in columns:
                
                if column == column2:
                    continue
                
                years = correlation_results.loc[column, (station_id, column2)].index
                
                correlation = correlation_results.loc[(column, years), (station_id, column2)].dropna()            
                
                fig.add_trace(
                    go.Scatter(
                        x=correlation.index.get_level_values(1),
                        y=correlation.apply(lambda x: x[0]),
                        mode='lines+markers',
                        name=column2
                    )
                )
                
            fig.update_layout(
                title=f"Station {station_id} - {column} vs Other Parameters",
                xaxis_title="Year",
                yaxis_title="Pearson Correlation Coefficient",
                # put legend inside the plot
                legend=dict(
                    x=0.01,
                    y=0.99
                )
            )
            
            
            fig.update_yaxes(range=[-1, 1])
            
            # add a horizontal line at 0
            fig.add_shape(
                type="line",
                x0=years.min(),
                y0=0,
                x1=years.max(),
                y1=0,
                line=dict(
                    color="black",
                    width=1,
                    dash="dashdot"
                )
            )
            
            if not os.path.exists(os.path.join(correlation_folder, f"station_{station_id}")):
                os.makedirs(os.path.join(correlation_folder, f"station_{station_id}"))
                
            column_ = column.replace("/", "_")
            
            fig.write_image(
                os.path.join(
                    correlation_folder,
                    f"station_{station_id}",
                    f"{column_}.png"
                ),
                scale=6,
                
            )

### Ground

In [ ]:
columns = ['Air Temperature (°C)', 'Ammonium (mg/l)', 'UVA254 (1/m)']

# perform year by year correlation
correlation_results = pd.DataFrame(
    index=pd.MultiIndex.from_product([columns, sorted(ground_df['DateTime'].dt.year.unique())]),
    columns=pd.MultiIndex.from_product([ground_df['Station'].unique().tolist(), columns])
)

for station_id in ground_df['Station'].unique():
    station_df = ground_df[ground_df['Station'] == station_id].copy()
    
    # normalize the data
    for column in columns:
        scaler = MinMaxScaler()
        station_df[column] = scaler.fit_transform(station_df[[column]])
    
    correlation_results[station_id] = {}
    
    for year in station_df['DateTime'].dt.year.unique():
        
        year_df = station_df[station_df['DateTime'].dt.year == year]
        
        year_df = year_df[columns]
        
        for column in year_df.columns:
            for column2 in year_df.columns:
                if column == column2:
                    continue
                
                result = pearsonr(year_df[column], year_df[column2])
                
                correlation_results.loc[(column, year), (station_id, column2)] = result

In [ ]:
# plot the correlation results

correlation_folder = os.path.join(paper_plots_folder, 'Berlin', 'Correlations', 'Ground')


for station_id in ground_df['Station'].unique():
        
        station_df = ground_df[ground_df['Station'] == station_id].copy()
        
        station_df.set_index('DateTime', inplace=True)
        
        for column in columns:
            
            fig = go.Figure()
            
            for column2 in columns:
                
                if column == column2:
                    continue
                
                years = correlation_results.loc[column, (station_id, column2)].index
                
                correlation = correlation_results.loc[(column, years), (station_id, column2)].dropna()            
                
                fig.add_trace(
                    go.Scatter(
                        x=correlation.index.get_level_values(1),
                        y=correlation.apply(lambda x: x[0]),
                        mode='lines+markers',
                        name=column2
                    )
                )
                
            fig.update_layout(
                title=f"Station {station_id} - {column} vs Other Parameters",
                xaxis_title="Year",
                yaxis_title="Pearson Correlation Coefficient",
                # put legend inside the plot
                legend=dict(
                    x=0.01,
                    y=0.99
                )
            )
            
            
            fig.update_yaxes(range=[-1, 1])
            
            # add a horizontal line at 0
            fig.add_shape(
                type="line",
                x0=years.min(),
                y0=0,
                x1=years.max(),
                y1=0,
                line=dict(
                    color="black",
                    width=1,
                    dash="dashdot"
                )
            )
            
            if not os.path.exists(os.path.join(correlation_folder, f"station_{station_id}")):
                os.makedirs(os.path.join(correlation_folder, f"station_{station_id}"))
                
            column_ = column.replace("/", "_")
            
            fig.write_image(
                os.path.join(
                    correlation_folder,
                    f"station_{station_id}",
                    f"{column_}.png"
                ),
                scale=6,
                
            )

## Special Case using all the available data

In [ ]:
# one plot for each variable
trend_folder = os.path.join(paper_plots_folder, 'Berlin', 'Trends', 'Overall')

# sort the columns into climatic, water quality and DOC as last one
climatic_variables = ['Air Temperature (°C)', 'Cumulated Rainfall (mm)']
water_quality_variables = ['Ammonium (mg/l)', 'Conductivity (µS/cm)', 'Dissolved Oxygen (mg/l)', 'Flow River Rate (m³/s)', 'Nitrate (mg/l)', 'pH', 'Water Temperature (°C)']
doc_variable = ['DOC (mg/l)']

columns = climatic_variables + water_quality_variables + doc_variable

colors = px.colors.qualitative.Plotly

# give me two colors for the trend lines different from the other colors

station_id = 325

station_df = surface_df[surface_df['Station'] == station_id].copy()

station_df.set_index('DateTime', inplace=True)

station_df = station_df.drop(columns=bacteria_columns)


statistics_df = pd.DataFrame(
    index=['MK Result', 'MK p-value', 'Slope', 'Slope p-value'],
    columns=pd.MultiIndex.from_product([['Pre 1998', 'Post 1998'], columns])
)

for i, column in enumerate(columns):
    
    fig = go.Figure()
    
    pre_wall_df = station_df[station_df.index < '1998-01-01']
    after_wall_df = station_df[station_df.index >= '1998-01-01']
    
    pre_df = pd.DataFrame({
        'ds': pre_wall_df.index,
        'y': pre_wall_df[column]
    })
    
    after_df = pd.DataFrame({
        'ds': after_wall_df.index,
        'y': after_wall_df[column]
    })
    
    pre_model = Prophet(weekly_seasonality=False, daily_seasonality=False)
    pre_model.fit(pre_df)
    # Make predictions for both columns
    pre_future = pre_model.make_future_dataframe(periods=0)
    pre_forecast = pre_model.predict(pre_future)
    
    
    
    after_model = Prophet(weekly_seasonality=False, daily_seasonality=False)
    after_model.fit(after_df)
    # Make predictions for both columns
    after_future = after_model.make_future_dataframe(periods=0)
    after_forecast = after_model.predict(after_future)
    
    
    # compute the Mann-Kendall test for the trend
    pre_result = mk.original_test(pre_wall_df[column])
    after_result = mk.original_test(after_wall_df[column])
    
    statistics_df.loc['MK Result', ('Pre 1998', column)] = pre_result.trend
    statistics_df.loc['MK Result', ('Post 1998', column)] = after_result.trend
    
    statistics_df.loc['MK p-value', ('Pre 1998', column)] = pre_result.p
    statistics_df.loc['MK p-value', ('Post 1998', column)] = after_result.p
    
    # compute the linear regression for the trend
    X = np.arange(pre_forecast.shape[0])
    X = sm.add_constant(X)
    y = pre_forecast['trend'].copy()
    
    pre_model = sm.OLS(y, X)
    pre_results = pre_model.fit()
    
    statistics_df.loc['Slope', ('Pre 1998', column)] = pre_results.params.iloc[1]
    statistics_df.loc['Slope p-value', ('Pre 1998', column)] = pre_results.pvalues.iloc[1]
    
    X = np.arange(after_forecast.shape[0])
    X = sm.add_constant(X)
    y = after_forecast['trend'].copy()
    
    after_model = sm.OLS(y, X)
    after_results = after_model.fit()
    
    statistics_df.loc['Slope', ('Post 1998', column)] = after_results.params.iloc[1]
    statistics_df.loc['Slope p-value', ('Post 1998', column)] = after_results.pvalues.iloc[1]
    

    fig.add_trace(
        go.Scatter(
            x=pre_wall_df.index,
            y=pre_wall_df[column],
            mode='lines',
            line=dict(
                color=colors[i],
                width=1.5
            ),
        ),
    )
    
    fig.add_trace(
        go.Scatter(
            x=pre_forecast['ds'],
            y=pre_forecast['trend'],
            mode='lines',
            name='Pre Wall Trend',
            line=dict(color=color, width=1),
        ),
    )
    
    fig.add_trace(
        go.Scatter(
            x=after_wall_df.index,
            y=after_wall_df[column],
            mode='lines',
            line=dict(
                color=colors[i],
                width=1.5
            ),
        ),
    )
    
    fig.add_trace(
        go.Scatter(
            x=after_forecast['ds'],
            y=after_forecast['trend'],
            mode='lines',
            name='After Wall Trend',
            line=dict(color=color, width=1),
        ),
    )
        
    # fig.update_yaxes(title_text=column)
    
    # Set y-axis range for a specific trace (e.g., the first trace)
    # if column == 'Ammonium (mg/l)':  # Replace with the actual column name
    #     fig.update_yaxes(range=[0, 0.7])
    
    # if column == 'DOC (mg/l)':
    #     fig.update_yaxes(range=[4, 12])
    
    # if column == 'Air Temperature (°C)':
    #     fig.update_yaxes(range=[-10, 30])
    
# # Add vertical dashed lines for each year
# years = station_df.index.year.unique()
# for year in years:
#     fig.add_vline(
#         x=pd.Timestamp(f'{year}-01-01'),
#         line=dict(color='gray', width=0.5, dash='dash'),
#         layer='above'
#     )
    
    fig.update_xaxes(title_text="Time")

    start_year = station_df.index.year.min()
    end_year = station_df.index.year.max()
    tickvals = [pd.Timestamp(f'{year}-01-01') for year in range(start_year, end_year + 1, 4)]
    ticktext = [str(year) for year in range(start_year, end_year + 1, 4)]   

    fig.update_xaxes(
        tickvals=tickvals,
        ticktext=ticktext,
        title_text="Time"
    )  
    
    fig.update_yaxes(title_text=column)

    fig.update_layout(
        legend=dict(
            traceorder='normal',
        ),
        showlegend=False,
        shapes=[
            dict(
                type="line",
                xref="x",
                yref="paper",
                x0=pd.Timestamp('1998-01-01'),
                y0=0,
                x1=pd.Timestamp('1998-01-01'),
                y1=1,
                line=dict(
                    color="red",
                    width=1,
                    dash="dash",
                ),
                layer="above"
            )
        ]
    )


    #reduce the font size of the subplot titles
    for annotation in fig['layout']['annotations']:
        annotation['font'] = dict(
            size=8,
            )

    # put the annotations above other elements
        
    column_ = column.replace("/", "_")

    fig.write_image(
        os.path.join(trend_folder, f"{column_}.png"),
        scale=15,
    )

    # fig.show()

statistics_df.to_excel(
    os.path.join(trend_folder, 'statistics.xlsx')
)


In [ ]:
%%script false --no-raise-error
trend_folder = os.path.join(paper_plots_folder, 'Berlin', 'Trends', 'Overall')

# sort the columns into climatic, water quality and DOC as last one
climatic_variables = ['Air Temperature (°C)', 'Cumulated Rainfall (mm)']
water_quality_variables = ['Ammonium (mg/l)', 'Conductivity (µS/cm)', 'Dissolved Oxygen (mg/l)', 'Flow River Rate (m³/s)', 'Nitrate (mg/l)', 'pH', 'Water Temperature (°C)']
doc_variable = ['DOC (mg/l)']

columns = climatic_variables + water_quality_variables + doc_variable

colors = px.colors.qualitative.Plotly

# give me two colors for the trend lines different from the other colors

station_id = 325

station_df = surface_df[surface_df['Station'] == station_id].copy()

station_df.set_index('DateTime', inplace=True)

station_df = station_df.drop(columns=bacteria_columns)


statistics_df = pd.DataFrame(
    index=['MK Result', 'MK p-value', 'Slope', 'Slope p-value'],
    columns=pd.MultiIndex.from_product([['Pre 1998', 'Post 1998'], columns])
)


fig = make_subplots(
    len(columns),
    1,
    shared_xaxes=True,
    subplot_titles=columns,
    vertical_spacing=0.02
)

trend_show = True

for i, column in enumerate(columns):
    
    pre_wall_df = station_df[station_df.index < '1998-01-01']
    after_wall_df = station_df[station_df.index >= '1998-01-01']
    
    pre_df = pd.DataFrame({
        'ds': pre_wall_df.index,
        'y': pre_wall_df[column]
    })
    
    after_df = pd.DataFrame({
        'ds': after_wall_df.index,
        'y': after_wall_df[column]
    })
    
    pre_model = Prophet(weekly_seasonality=False, daily_seasonality=False)
    pre_model.fit(pre_df)
    # Make predictions for both columns
    pre_future = pre_model.make_future_dataframe(periods=0)
    pre_forecast = pre_model.predict(pre_future)
    
    
    
    after_model = Prophet(weekly_seasonality=False, daily_seasonality=False)
    after_model.fit(after_df)
    # Make predictions for both columns
    after_future = after_model.make_future_dataframe(periods=0)
    after_forecast = after_model.predict(after_future)
    
    
    # compute the Mann-Kendall test for the trend
    pre_result = mk.original_test(pre_wall_df[column])
    after_result = mk.original_test(after_wall_df[column])
    
    statistics_df.loc['MK Result', ('Pre 1998', column)] = pre_result.trend
    statistics_df.loc['MK Result', ('Post 1998', column)] = after_result.trend
    
    statistics_df.loc['MK p-value', ('Pre 1998', column)] = pre_result.p
    statistics_df.loc['MK p-value', ('Post 1998', column)] = after_result.p
    
    # compute the linear regression for the trend
    X = np.arange(pre_forecast.shape[0])
    X = sm.add_constant(X)
    y = pre_forecast['trend'].copy()
    
    pre_model = sm.OLS(y, X)
    pre_results = pre_model.fit()
    
    statistics_df.loc['Slope', ('Pre 1998', column)] = pre_results.params.iloc[1]
    statistics_df.loc['Slope p-value', ('Pre 1998', column)] = pre_results.pvalues.iloc[1]
    
    X = np.arange(after_forecast.shape[0])
    X = sm.add_constant(X)
    y = after_forecast['trend'].copy()
    
    after_model = sm.OLS(y, X)
    after_results = after_model.fit()
    
    statistics_df.loc['Slope', ('Post 1998', column)] = after_results.params.iloc[1]
    statistics_df.loc['Slope p-value', ('Post 1998', column)] = after_results.pvalues.iloc[1]
    

    fig.add_trace(
        go.Scatter(
            x=pre_wall_df.index,
            y=pre_wall_df[column],
            mode='lines',
            line=dict(
                color=colors[i],
                width=1.5
            ),
            legendrank=i,
            showlegend=False
        ),
        row=i + 1,
        col=1
    )
    
    fig.add_trace(
        go.Scatter(
            x=pre_forecast['ds'],
            y=pre_forecast['trend'],
            mode='lines',
            name='Pre Wall Trend',
            line=dict(color=color, width=1),
            showlegend=False, # change to trend_show if you want to show the legend
            legendrank=np.inf
        ),
        row=i + 1,
        col=1
    )
    
    fig.add_trace(
        go.Scatter(
            x=after_wall_df.index,
            y=after_wall_df[column],
            mode='lines',
            line=dict(
                color=colors[i],
                width=1.5
            ),
            legendrank=i,
            showlegend=False
        ),
        row=i + 1,
        col=1
    )
    
    fig.add_trace(
        go.Scatter(
            x=after_forecast['ds'],
            y=after_forecast['trend'],
            mode='lines',
            name='After Wall Trend',
            line=dict(color=color, width=1),
            showlegend=False, # change to trend_show if you want to show the legend
            legendrank=np.inf
        ),
        row=i + 1,
        col=1
    )
    
    if trend_show:
        trend_show = False
        
    # fig.update_yaxes(title_text=column, row=i + 1, col=1)
    
    # Set y-axis range for a specific trace (e.g., the first trace)
    # if column == 'Ammonium (mg/l)':  # Replace with the actual column name
    #     fig.update_yaxes(range=[0, 0.7], row=i + 1, col=1)
    
    # if column == 'DOC (mg/l)':
    #     fig.update_yaxes(range=[4, 12], row=i + 1, col=1)
    
    # if column == 'Air Temperature (°C)':
    #     fig.update_yaxes(range=[-10, 30], row=i + 1, col=1)
    
# # Add vertical dashed lines for each year
# years = station_df.index.year.unique()
# for year in years:
#     fig.add_vline(
#         x=pd.Timestamp(f'{year}-01-01'),
#         line=dict(color='gray', width=0.5, dash='dash'),
#         layer='above'
#     )
    
fig.update_xaxes(title_text="Time", row=len(columns), col=1)

start_year = station_df.index.year.min()
end_year = station_df.index.year.max()
tickvals = [pd.Timestamp(f'{year}-01-01') for year in range(start_year, end_year + 1, 4)]
ticktext = [str(year) for year in range(start_year, end_year + 1, 4)]   

fig.update_xaxes(
    tickvals=tickvals,
    ticktext=ticktext,
)  

fig.update_layout(
    font=dict(size=8),
    legend=dict(
        traceorder='normal',
    ),
    shapes=[
        dict(
            type="line",
            xref="x",
            yref="paper",
            x0=pd.Timestamp('1998-01-01'),
            y0=0,
            x1=pd.Timestamp('1998-01-01'),
            y1=1,
            line=dict(
                color="red",
                width=1,
                dash="dash",
            ),
            layer="above"
        )
    ],
    margin=dict(
        l=30,  # Left margin
        r=30,  # Right margin
        t=50,  # Top margin
        b=150  # Bottom margin (increase to add blank space)
    )
)


#reduce the font size of the subplot titles
for annotation in fig['layout']['annotations']:
    annotation['font'] = dict(
        size=8,
        )

# put the annotations above other elements
    


fig.write_image(
    os.path.join(trend_folder, f"station_{station_id}.png"),
    scale=15,
    width=400,
    height=220 * len(columns)
)


statistics_df.to_excel(
    os.path.join(trend_folder, 'statistics.xlsx')
)
